# Building Fast Grammar Fuzzers

## System configuration

In [ ]:
!uname -v

In [ ]:
!sw_vers

In [ ]:
!system_profiler SPHardwareDataType

In [ ]:
import pkg_resources
def ipkg(pkg, repo):
    try:
        distinfo = pkg_resources.get_distribution(pkg)
    except pkg_resources.DistributionNotFound:
        !pip install {repo}
    else:
        print(pkg, 'found')

In [ ]:
ipkg('fuzzingbook', 'pip install fuzzingbook')

In [ ]:
from fuzzingbook.Timer import Timer
from fuzzingbook.ExpectError import ExpectTimeout

In [ ]:
!rm -rf /tmp/stop.ffg
!rm -rf testers tests
!mkdir -p testers tests

In [ ]:
!rm -rf results

**important** We rely on a relatively high recursion limit : 20900 which is only available on MacOSX (Not in Linux).

## Why focus on faster grammar fuzzing?
* Guided fuzzing of uninstrumented code (aka. memory fuzzing).
* Grammar Mining advances

## Building a simple fuzzer

In [ ]:
import random
import string
import statistics

In [ ]:
def producer(chars, l, n=1):
    return [''.join([random.choice(chars) for i in range(l)]) for i in range(n)]

In [ ]:
producer(string.printable, 100)

In [ ]:
import os, subprocess
from datetime import datetime

## A better tester

In [ ]:
import os, subprocess
from datetime import datetime
from resource import getrusage as resource_usage, RUSAGE_CHILDREN
from time import time as timestamp
START_TIME = datetime.now()

In [ ]:
class timeit():
    def __init__(self):
        pass
    def __enter__(self):
        self.start_time, self.start_resources = timestamp(), resource_usage(RUSAGE_CHILDREN)
        return self
    def __exit__(self, *args, **kwargs):
        end_time, end_resources = timestamp(), resource_usage(RUSAGE_CHILDREN)
        self._runtime = end_time - self.start_time
        self.sys_runtime = end_resources.ru_stime - self.start_resources.ru_stime 
        self.usr_runtime = end_resources.ru_utime - self.start_resources.ru_utime
        self.runtime = self.sys_runtime + self.usr_runtime

In [ ]:
TX = {}

In [ ]:
# II
class Tester:
    def __init__(self, name=None, max_num=10000, start_depth=3, limit_depth=9, timeout=3600, iterations=2):
        global TX
        if name is not None:
            self.tname = name
        else:
            self.tname = self.__class__.__name__
        self.tx = TX
        self.max_num, self.start_depth, self.limit_depth, self.timeout, self.iterations = \
            max_num, start_depth, limit_depth, timeout, iterations
        self.tst = {}
        self.tx[self.tname] = self.tst
        self.WARMUP_TIMES = 10
        self.timedout = None
        
    def write_t(self, cmd):
        self.t = "testers/%s-t.sh" % self.tname
        with open(self.t, 'w') as f:
            print('''\
#!/usr/bin/env bash
TIMEFORMAT="%%U %%S";
time %(cmd)s''' % {'cmd':cmd}, file=f)
        !chmod +x {self.t}
        
        
    def init_run(self):
        !rm -rf testers/{self.tname}
        !mkdir -p testers/{self.tname}

    def pre_time(self):
        !rm -rf tests
        !mkdir -p tests
        
    def pre_exec(self, t):
        pass
        
    def exec_program(self, seed, max_depth, t):
        raise NotImplementedError()

    def post_exec(self, t):
        pass
    
    def post_time(self):
        if not self._runtime: return
        if self.file is not None and os.path.exists(self.file):
            lines_cmd = ("cat %s| wc -l" % self.file)
            self.lines = subprocess.getoutput(lines_cmd).strip()
            #unique_cmd = ("cat %s| sort -u| wc -l" % self.file)
            #self.unique_lines = subprocess.getoutput(unique_cmd).strip()
            self.size = os.stat(self.file).st_size
        else:
            self.unique_lines = ''
            self.lines = ''
            self.size = 0
        self.throughput = (self.size/1024/self._runtime, self._runtime)
    
    def timed_exec(self, seed, max_depth, verbose):
        self.pre_time()
        self._runtime = None
        self.timedout = True
        with ExpectTimeout(self.timeout): #, print_traceback=False, mute=True):
            #with timeit() as t:
            t = None
            self.pre_exec(t)
            cmdline = self.exec_program(seed, max_depth, t)
            self.write_t(cmdline)
            !{self.t} 2>./testers/time.out
            self.post_exec(t)
            with open('testers/time.out') as f:
                usr, sys = f.read().strip().split(' ')
            self._runtime = float(usr)+ float(sys)
            self._sys_runtime = float(sys)
            self._usr_runtime = float(usr)
            self.timedout = False
        self.post_time()

    def ofile(self, max_depth, seed):
        fn = 'testers/%s/%d_%d.x' % (self.tname, max_depth, seed)
        return fn
    
    def check_continue(self):
        if os.path.exists('/tmp/stop.ffg'):
            raise Exception('/tmp/stop.ffg -- abort tests')

    def run_test(self, verbose=False):
        def warmup(seed):
            # for warming up, we simply run it a few times before in the
            # same seed as the first, and discount it in computation.
            return [seed]*self.WARMUP_TIMES
        current_time = datetime.now()
        self.init_run()
        # depth is for later when we deal with grammars.
        
        # warmup loop
        for md in [self.start_depth]:
            max_depth = 2**md
            for seed in warmup(0):
                self.file = self.ofile(max_depth, seed)
                self.timed_exec(seed, max_depth, verbose)
                if os.path.exists(self.file): os.remove(self.file)
                
        for md in range(self.start_depth, self.limit_depth):
            max_depth = 2**md
            v = {}
            res = {'detail': v}
            self.tst[max_depth] = res
            seeds = list(range(self.iterations))
            for seed in seeds:
                if self.timedout: break
                self.file = self.ofile(max_depth, seed)
                if verbose: print('depth:', max_depth, 'seed:', seed, 'file:', self.file)
                self.timed_exec(seed, max_depth, verbose)
                if self._runtime:
                    v[seed] = {
                        'runtime':self._runtime,
                        'sys_runtime':self._sys_runtime,
                        'usr_runtime':self._usr_runtime,
                        'size': self.size,
                        #'uniq': self.unique_lines,
                        'lines': self.lines,
                        # in kbytes
                        'throughput': self.size/self._runtime/(1024)}
                if verbose:
                    print(v[seed])
                if os.path.exists(self.file): os.remove(self.file)
                self.check_continue()
            if self.timedout:
                print('Timeout')
                break # we do not expect larger depths to work.
            size = [t['size'] for t in v.values()]
            res['avgsize'] = statistics.mean(size)
                
            sec = [t['runtime'] for t in v.values()]
            res['avgruntime'] = statistics.mean(sec)
            res['stdevruntime'] = statistics.stdev(sec)
                
            tp = [t['throughput'] for t in v.values()]
            res['avgthroughput'] = statistics.mean(tp)
            res['stdevthroughput'] = statistics.stdev(tp)
            print('depth=', max_depth, "size=", res['avgsize'], 'time=', round(res['avgruntime'],3), "stdev(%s)" % str(round(res['stdevruntime'],3)), 'throughput=',res['avgthroughput'], "stdev(%s)" % str(round(res['stdevthroughput'])))
        self.total_test_time = datetime.now() - current_time
        self.dump()
        return self
    
    def dump(self):
        curtime = datetime.now().isoformat()
        name = 'results/%s-tx.json' % (self.tname)
        !mkdir -p results
        with open(name, 'w+') as f:
            print(json.dumps(TX), file=f)
    
    def show(self):
        max_throughput = 0
        best_depth = None
        for depth in self.tst.keys():
            res = self.tst[depth]
            if res.get('avgthroughput',0) > max_throughput:
                max_throughput = res['avgthroughput'] 
                best_depth = depth
        print('Throughput of ', max_throughput, ' kilobytes per second at depth = ', best_depth)
        print("Total time:",str(self.total_test_time))

In [ ]:
class RandomTester(Tester):
    def pre_time(self):
        with open('testers/RandomTester/r.py', 'w+') as f:
            print('''
import string,random,sys
random.seed(int(sys.argv[1]))
def producer(chars, l, n=1):
    return [''.join([random.choice(chars) for i in range(l)]) for i in range(n)]
print(producer(string.printable, int(sys.argv[2])))''', file=f)
            
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"python testers/RandomTester/r.py {seed} {max_depth} > {fn}"

In [ ]:
RandomTester().run_test().show()

In [ ]:
TX

## Grammars

In [ ]:
# from fuzzingbook.Parser import make_grammar
# We need a bit of modification make the grammars more varied

In [ ]:
def prod_line_grammar(nonterminals, terminals):
    g = {
        '<start>': ['<symbols>'],
        '<symbols>': ['<symbol><symbols>', '<symbol>'],
        '<symbol>': ['<nonterminals>', '<terminals>'],
        '<nonterminals>': ['<lt><alpha><gt>'],
        '<lt>': ['<'],
        '<gt>': ['>'],
        '<alpha>': nonterminals,
        '<terminals>': terminals
    }

    if not nonterminals:
        g['<nonterminals>'] = ['']
        del g['<lt>']
        del g['<alpha>']
        del g['<gt>']

    return g


In [ ]:
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
from fuzzingbook.Parser import canonical
from fuzzingbook.Grammars import unreachable_nonterminals, RE_NONTERMINAL
import random, string, re

In [ ]:
def make_rule(nonterminals, terminals, num_alts):
    prod_grammar = prod_line_grammar(nonterminals, terminals)

    gf = GrammarFuzzer(prod_grammar, min_nonterminals=3, max_nonterminals=5)
    name = "<%s>" % ''.join(random.choices(string.ascii_uppercase, k=3))

    return (name, [gf.fuzz() for _ in range(num_alts)])

In [ ]:
make_rule(["A", "B", "C"], ["1", "2", "3"], 3)

In [ ]:
def make_grammar(num_symbols=3, num_alts=3):
    terminals = list(string.ascii_lowercase)
    grammar = {}
    name = None
    for _ in range(num_symbols):
        nonterminals = [k[1:-1] for k in grammar.keys()]
        name, expansions = \
            make_rule(nonterminals, terminals, num_alts)
        grammar[name] = expansions

    grammar['<start>'] = [name]

    # Remove unused parts
    for nonterminal in unreachable_nonterminals(grammar):
        del grammar[nonterminal]
        
    return grammar

In [ ]:
canonical(make_grammar())

### A CSS grammar

In [ ]:
css_grammar = {
    '<start>': [['<stylesheet>']],
    '<stylesheet>': [[
        '<[CHARSET_SYM_STRING_SEMI]-1>', ' ', '<[S_OR_CDO_OR_CDC]-1>', ' ',
        '<[import_CDO_S_OR_CDC_S]-1>', ' ', '<[stylesheet_closing_GROUPING]-1>'
    ]],
    '<[CHARSET_SYM_STRING_SEMI]>': [['<CHARSET_SYM>', ' ', '<STRING>', ' ;']],
    '<[S_OR_CDO_OR_CDC]>': [['<Sp>'], ['<CDO>'], ['<CDC>']],
    '<[import_CDO_S_OR_CDC_S]>': [['<import>', ' ', '<[CDO_S_OR_CDC_S]-1>']],
    '<[CDO_S_OR_CDC_S]>': [['<CDO>', ' ', '<Ss>'], ['<CDC>', ' ', '<Ss>']],
    '<[ruleset_OR_media_OR_page]>': [['<ruleset>'], ['<media>'], ['<page>']],
    '<[stylesheet_closing_GROUPING]>':
    [['<[ruleset_OR_media_OR_page]>', ' ', '<[CDO_S_OR_CDC_S]-2>']],
    '<import>': [[
        '<IMPORT_SYM>', ' ', '<Ss>', ' ', '<[STRING_OR_URI]>', ' ', '<Ss>',
        ' ', '<media_list-1>', ' ; ', '<Ss>'
    ]],
    '<[STRING_OR_URI]>': [['<STRING>'], ['<URI>']],
    '<media>': [[
        '<MEDIA_SYM>', ' ', '<Ss>', ' ', '<media_list>', ' { ', '<Ss>', ' ',
        '<ruleset-1>', ' } ', '<Ss>'
    ]],
    '<media_list>': [['<medium>', ' ', '<[COMMA_S_medium]-1>']],
    '<[COMMA_S_medium]>': [[', ', '<Ss>', ' ', '<medium>']],
    '<medium>': [['<IDENT>', ' ', '<Ss>']],
    '<page>': [[
        '<PAGE_SYM>', ' ', '<Ss>', ' ', '<pseudo_page-1>', ' { ', '<Ss>', ' ',
        '<declaration-1>', ' ', '<[SEMI_S_declaration]-1>', ' } ', '<Ss>'
    ]],
    '<[SEMI_S_declaration]>': [['; ', '<Ss>', ' ', '<declaration-2>']],
    '<pseudo_page>': [[': ', '<IDENT>', ' ', '<Ss>']],
    '<operator>': [['/ ', '<Ss>'], [', ', '<Ss>']],
    '<combinator>': [['+ ', '<Ss>'], ['> ', '<Ss>']],
    '<unary_operator>': [['-'], ['+']],
    '<property>': [['<IDENT>', ' ', '<Ss>']],
    '<ruleset>': [[
        '<selector>', ' ', '<COMMA_S_selector-1>', ' { ', '<Ss>', ' ',
        '<declaration-3>', ' ', '<[SEMI_S_declaration]-2>', ' } ', '<Ss>'
    ]],
    '<COMMA_S_selector>': [[', ', '<Ss>', ' selector']],
    '<selector>':
    [['<simple_selector>', ' ', '<[combinator_selector_OR_S]-1>']],
    '<[combinator_selector]>': [['<combinator-1>', ' ', '<selector>']],
    '<[combinator_selector_OR_S]>':
    [['<combinator>', ' ', '<selector>'],
     ['<Sp>', ' ', '<[combinator_selector]-1>']],
    '<simple_selector>':
    [['<element_name>', ' ', '<[HASH_OR_class_OR_attrib_OR_pseudo]-1>'],
     ['<[HASH_OR_class_OR_attrib_OR_pseudo]-2>']],
    '<[HASH_OR_class_OR_attrib_OR_pseudo]>': [['<HASH>'], ['<class>'],
                                              ['<attrib>'], ['<pseudo>']],
    '<class>': [['.', '<IDENT>']],
    '<element_name>': [['<IDENT>'], ['*']],
    '<attrib>': [[
        '[ ', '<Ss>', ' ', '<IDENT>', ' ', '<Ss>', ' ',
        '<[attrib_GROUPING]-1>', ' ]'
    ]],
    '<[EQUAL_OR_INCLUDES_OR_DASHMATCH]>': [['='], ['<INCLUDES>'],
                                           ['<DASHMATCH>']],
    '<[IDENT_OR_STRING]>': [['<IDENT>'], ['<STRING>']],
    '<[attrib_GROUPING]>': [[
        '<[EQUAL_OR_INCLUDES_OR_DASHMATCH]>', ' ', '<Ss>', ' ',
        '<[IDENT_OR_STRING]>', ' ', '<Ss>'
    ]],
    '<pseudo>': [[': ', '<[IDENT_OR_FUNCTION]>']],
    '<[IDENT_OR_FUNCTION]>':
    [['<IDENT>'], ['<FUNCTION>', ' ', '<Ss>', ' ', '<[IDENT_S]-1>', ' )']],
    '<[IDENT_S]>': [['<IDENT>', ' ', '<Ss>']],
    '<declaration>':
    [['<property>', ' : ', '<Ss>', ' ', '<expr>', ' ', '<prio-1>']],
    '<prio>': [['<IMPORT_SYM>', ' ', '<Ss>']],
    '<expr>': [['<term>', ' ', '<[operator_term]-1>']],
    '<[operator_term]>': [['<operator-1>', ' ', '<term>']],
    '<term>': [['<unary_operator-1>', ' ', '<[term_GROUPING]>'],
               ['<STRING>', ' ', '<Ss>'], ['<IDENT>', ' ', '<Ss>'],
               ['<URI>', ' ', '<Ss>'], ['<hexcolor>'], ['<function>']],
    '<[term_GROUPING]>': [['<NUMBER>', ' ', '<Ss>'],
                          ['<PERCENTAGE>', ' ', '<Ss>'],
                          ['<LENGTH>', ' ', '<Ss>'], ['<EMS>', ' ', '<Ss>'],
                          ['<EXS>', ' ', '<Ss>'], ['<ANGLE>', ' ', '<Ss>'],
                          ['<TIME>', ' ', '<Ss>'], ['<FREQ>', ' ', '<Ss>']],
    '<function>': [['<FUNCTION>', ' ', '<Ss>', ' ', '<expr>', ' ) ', '<Ss>']],
    '<hexcolor>': [['#', '<[three_char_HEX]>', ' ', '<Ss>'],
                   ['#', '<[six_char_HEX]>', ' ', '<Ss>']],
    '<[three_char_HEX]>': [['<HEX_CHAR>', '<HEX_CHAR>', '<HEX_CHAR>']],
    '<[six_char_HEX]>': [[
        '<HEX_CHAR>', '<HEX_CHAR>', '<HEX_CHAR>', '<HEX_CHAR>', '<HEX_CHAR>',
        '<HEX_CHAR>'
    ]],
    '<HEX_CHAR>': [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'],
                   ['8'], ['9'], ['a'], ['b'], ['c'], ['d'], ['e'], ['f']],
    '<Sp>': [['<WHITESPACE-1>']],
    '<Ss>': [['<WHITESPACE-2>']],
    '<So>': [[], ['<WHITESPACE>']],
    '<CDO>': [['<!--']],
    '<CDC>': [['-->']],
    '<INCLUDES>': [['~=']],
    '<DASHMATCH>': [['|=']],
    '<STRING>': [['<string1>'], ['<string2>']],
    '<IDENT>': [['<{ident}>']],
    '<HASH>': [['#', '<{name}>']],
    '<IMPORT_SYM>': [['@import']],
    '<PAGE_SYM>': [['@page']],
    '<MEDIA_SYM>': [['@media']],
    '<CHARSET_SYM>': [['@charset']],
    '<EMS>': [['<{num}>', 'em']],
    '<EXS>': [['<{num}>', 'ex']],
    '<LENGTH>': [['<{num}>', 'px'], ['<{num}>', 'cm'], ['<{num}>', 'mm'],
                 ['<{num}>', 'in'], ['<{num}>', 'pt'], ['<{num}>', 'pc']],
    '<ANGLE>': [['<{num}>', 'deg'], ['<{num}>', 'rad'], ['<{num}>', 'grad']],
    '<TIME>': [['<{num}>', 'ms'], ['<{num}>', 's']],
    '<FREQ>': [['<{num}>', 'hz'], ['<{num}>', 'khz']],
    '<PERCENTAGE>': [['<{num}>', '%']],
    '<NUMBER>': [['<{num}>']],
    '<URI>': [['url("', '<So>', '<url>', '<So>', '")'],
              ['url("', '<So>', '<STRING>', '<So>', '")']],
    '<url>': [['<url_-1>']],
    '<url_>': [['!'], ['#'], ['$'], ['%'], ['&'], ['*'], ['-'], ['~'],
               ['<escape>']],
    '<FUNCTION>': [['<{ident}>', '(']],
    '<string1>': [['"', '<qmychars1-1>', '"']],
    '<string2>': [["'", '<qmychars2-1>', "'"]],
    '<qnonl1>': [['7'], ['Q'], ['J'], ['@'], ['2'], ['g'], ['\t'], ['X'],
                 ['`'], ['G'], ['e'], ['['], ['?'], ['v'], ['$'], ['j'], ['K'],
                 ['d'], ['A'], ['n'], ['h'], ['l'], ['4'], ['D'], ['a'], ['#'],
                 ['f'], ['y'], ['B'], ['U'], ['P'], ['3'], ['O'], ['S'], [')'],
                 [' '], ['W'], ['o'], ['b'], ['|'], ['q'], ['L'], [']'], ['V'],
                 ['*'], ['z'], ['}'], ['6'], ['u'], ['^'], [','], ['N'], ['>'],
                 ['+'], ['Y'], ['t'], ['k'], ['!'], ['p'], ['Z'], ['E'], ['('],
                 ['\\'], ['<'], ['F'], ['%'], ['9'], ['0'], ['s'], [';'],
                 ['&'], ['C'], ['T'], ['r'], ['5'], ['R'], ["'"], ['_'], ['.'],
                 ['8'], ['H'], ['i'], ['/'], ['M'], ['~'], ['{'], [':'], ['c'],
                 ['I'], ['-'], ['\x0b'], ['1'], ['w'], ['x'], ['m'], ['=']],
    '<qnonl2>': [['7'], ['Q'], ['J'], ['@'], ['2'], ['g'], ['\t'], ['X'],
                 ['`'], ['G'], ['e'], ['['], ['?'], ['v'], ['$'], ['j'], ['K'],
                 ['d'], ['A'], ['n'], ['h'], ['l'], ['4'], ['D'], ['a'], ['#'],
                 ['f'], ['y'], ['B'], ['U'], ['P'], ['3'], ['O'], ['S'], [')'],
                 [' '], ['W'], ['o'], ['b'], ['|'], ['q'], ['L'], [']'], ['V'],
                 ['*'], ['z'], ['}'], ['6'], ['u'], ['^'], [','], ['N'], ['>'],
                 ['+'], ['Y'], ['t'], ['k'], ['!'], ['p'], ['Z'], ['E'], ['('],
                 ['\\'], ['<'], ['F'], ['%'], ['9'], ['0'], ['s'], [';'],
                 ['&'], ['C'], ['T'], ['r'], ['5'], ['"'], ['R'], ['_'], ['.'],
                 ['8'], ['H'], ['i'], ['/'], ['M'], ['~'], ['{'], [':'], ['c'],
                 ['I'], ['-'], ['\x0b'], ['1'], ['w'], ['x'], ['m'], ['=']],
    '<qmychars1>': [['<qnonl1>'], ['\\', '<nl>'], ['<escape>']],
    '<qmychars2>': [['<qnonl2>'], ['\\', '<nl>'], ['<escape>']],
    '<nl>': [['\r'], ['\n'], ['\x0c'], ['\r\n']],
    '<escape>': [['\\', '<echar>']],
    '<echar>': [['Q'], ['J'], ['@'], ['g'], ['\t'], ['X'], ['`'], ['G'], ['['],
                ['?'], ['v'], ['$'], ['j'], ['K'], ['A'], ['n'], ['h'], ['l'],
                ['D'], ['#'], ['y'], ['B'], ['U'], ['P'], ['O'], ['S'], [')'],
                [' '], ['W'], ['o'], ['|'], ['q'], ['L'], [']'], ['V'], ['*'],
                ['z'], ['}'], ['u'], ['^'], [','], ['N'], ['>'], ['+'], ['Y'],
                ['t'], ['k'], ['!'], ['p'], ['Z'], ['E'], ['('], ['\\'], ['<'],
                ['F'], ['%'], ['s'], [';'], ['&'], ['C'], ['T'], ['r'], ['"'],
                ['R'], ["'"], ['_'], ['.'], ['H'], ['i'], ['/'], ['M'], ['~'],
                ['{'], [':'], ['I'], ['-'], ['\x0b'], ['w'], ['x'], ['m'],
                ['=']],
    '<{ident}>': [['<minus-1>', '<nmstart>', '<nmchar-1>']],
    '<nmstart>': [['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['g'], ['h'],
                  ['i'], ['j'], ['k'], ['l'], ['m'], ['n'], ['o'], ['p'],
                  ['q'], ['r'], ['s'], ['t'], ['u'], ['v'], ['w'], ['x'],
                  ['y'], ['z'], ['<escape>'], ['_']],
    '<nmchar>': [['a'], ['b'], ['c'], ['d'], ['e'], ['f'], ['g'], ['h'], ['i'],
                 ['j'], ['k'], ['l'], ['m'], ['n'], ['o'], ['p'], ['q'], ['r'],
                 ['s'], ['t'], ['u'], ['v'], ['w'], ['x'], ['y'], ['z'], ['0'],
                 ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'],
                 ['<escape>'], ['_'], ['-']],
    '<minus>': [['-']],
    '<{name}>': [['<nmchar-2>']],
    '<{num}>': [['<INTEGER>']],
    '<WHITESPACE>': [[' '], ['\t']],
    '<INTEGER>': [['<DIGIT>', '<INTEGER>'], ['<DIGIT>']],
    '<DIGIT>': [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'],
                ['9']],
    '<[CHARSET_SYM_STRING_SEMI]-1>': [[], ['<[CHARSET_SYM_STRING_SEMI]>']],
    '<[S_OR_CDO_OR_CDC]-1>': [[],
                              ['<[S_OR_CDO_OR_CDC]>',
                               '<[S_OR_CDO_OR_CDC]-1>']],
    '<[import_CDO_S_OR_CDC_S]-1>':
    [[], ['<[import_CDO_S_OR_CDC_S]>', '<[import_CDO_S_OR_CDC_S]-1>']],
    '<[stylesheet_closing_GROUPING]-1>':
    [[],
     ['<[stylesheet_closing_GROUPING]>', '<[stylesheet_closing_GROUPING]-1>']],
    '<[CDO_S_OR_CDC_S]-1>': [[],
                             ['<[CDO_S_OR_CDC_S]>', '<[CDO_S_OR_CDC_S]-1>']],
    '<[CDO_S_OR_CDC_S]-2>': [[],
                             ['<[CDO_S_OR_CDC_S]>', '<[CDO_S_OR_CDC_S]-2>']],
    '<media_list-1>': [[], ['<media_list>']],
    '<ruleset-1>': [[], ['<ruleset>', '<ruleset-1>']],
    '<[COMMA_S_medium]-1>': [[],
                             ['<[COMMA_S_medium]>', '<[COMMA_S_medium]-1>']],
    '<pseudo_page-1>': [[], ['<pseudo_page>']],
    '<declaration-1>': [[], ['<declaration>']],
    '<[SEMI_S_declaration]-1>':
    [[], ['<[SEMI_S_declaration]>', '<[SEMI_S_declaration]-1>']],
    '<declaration-2>': [[], ['<declaration>']],
    '<COMMA_S_selector-1>': [[],
                             ['<COMMA_S_selector>', '<COMMA_S_selector-1>']],
    '<declaration-3>': [[], ['<declaration>']],
    '<[SEMI_S_declaration]-2>':
    [[], ['<[SEMI_S_declaration]>', '<[SEMI_S_declaration]-2>']],
    '<[combinator_selector_OR_S]-1>': [[], ['<[combinator_selector_OR_S]>']],
    '<combinator-1>': [[], ['<combinator>']],
    '<[combinator_selector]-1>': [[], ['<[combinator_selector]>']],
    '<[HASH_OR_class_OR_attrib_OR_pseudo]-1>':
    [[],
     [
         '<[HASH_OR_class_OR_attrib_OR_pseudo]>',
         '<[HASH_OR_class_OR_attrib_OR_pseudo]-1>'
     ]],
    '<[HASH_OR_class_OR_attrib_OR_pseudo]-2>':
    [['<[HASH_OR_class_OR_attrib_OR_pseudo]>'],
     [
         '<[HASH_OR_class_OR_attrib_OR_pseudo]>',
         '<[HASH_OR_class_OR_attrib_OR_pseudo]-2>'
     ]],
    '<[attrib_GROUPING]-1>': [[], ['<[attrib_GROUPING]>']],
    '<[IDENT_S]-1>': [[], ['<[IDENT_S]>']],
    '<prio-1>': [[], ['<prio>']],
    '<[operator_term]-1>': [[], ['<[operator_term]>', '<[operator_term]-1>']],
    '<operator-1>': [[], ['<operator>']],
    '<unary_operator-1>': [[], ['<unary_operator>']],
    '<WHITESPACE-1>': [['<WHITESPACE>'], ['<WHITESPACE>', '<WHITESPACE-1>']],
    '<WHITESPACE-2>': [[], ['<WHITESPACE>', '<WHITESPACE-2>']],
    '<url_-1>': [[], ['<url_>', '<url_-1>']],
    '<qmychars1-1>': [[], ['<qmychars1>', '<qmychars1-1>']],
    '<qmychars2-1>': [[], ['<qmychars2>', '<qmychars2-1>']],
    '<minus-1>': [[], ['<minus>']],
    '<nmchar-1>': [[], ['<nmchar>', '<nmchar-1>']],
    '<nmchar-2>': [['<nmchar>'], ['<nmchar>', '<nmchar-2>']]
}

### A JSON grammar

In [ ]:
json_grammar = {
    '<start>': [['<json>']],
    '<json>': [['<element>']],
    '<element>': [['<ws>', '<value>', '<ws>']],
    '<value>': [['<object>'], ['<array>'], ['<string>'], ['<number>'],
                ['true'], ['false'],
                ['null']],
    '<object>': [['{', '<ws>', '}'], ['{', '<members>', '}']],
    '<members>': [['<member>', '<symbol-2>']],
    '<member>': [['<ws>', '<string>', '<ws>', ':', '<element>']],
    '<array>': [['[', '<ws>', ']'], ['[', '<elements>', ']']],
    '<elements>': [['<element>', '<symbol-1-1>']],
    '<string>': [['"', '<characters>', '"']],
    '<characters>': [['<character-1>']],
    '<character>': [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'],
                    ['8'], ['9'], ['a'], ['b'], ['c'], ['d'], ['e'], ['f'],
                    ['g'], ['h'], ['i'], ['j'], ['k'], ['l'], ['m'], ['n'],
                    ['o'], ['p'], ['q'], ['r'], ['s'], ['t'], ['u'], ['v'],
                    ['w'], ['x'], ['y'], ['z'], ['A'], ['B'], ['C'], ['D'],
                    ['E'], ['F'], ['G'], ['H'], ['I'], ['J'], ['K'], ['L'],
                    ['M'], ['N'], ['O'], ['P'], ['Q'], ['R'], ['S'], ['T'],
                    ['U'], ['V'], ['W'], ['X'], ['Y'], ['Z'], ['!'], ['#'],
                    ['$'], ['%'], ['&'], ['\''], ['('], [')'], ['*'], ['+'],
                    [','], ['-'], ['.'], ['/'], [':'], [';'], ['<'], ['='],
                    ['>'], ['?'], ['@'], ['['], [']'], ['^'], ['_'], ['`'],
                    ['{'], ['|'], ['}'], ['~'], [' '], ['<esc>']],
    '<esc>': [['\\','<escc>']],
    '<escc>': [['\\'],['b'],['f'], ['n'], ['r'],['t'],['"']],
    '<number>': [['<int>', '<frac>', '<exp>']],
    '<int>': [['<digit>'], ['<onenine>', '<digits>'], ['-', '<digits>'],
              ['-', '<onenine>', '<digits>']],
    '<digits>': [['<digit-1>']],
    '<digit>': [['0'], ['<onenine>']],
    '<onenine>': [['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'],
                  ['9']],
    '<frac>': [[], ['.', '<digits>']],
    '<exp>': [[], ['E', '<sign>', '<digits>'], ['e', '<sign>', '<digits>']],
    '<sign>': [[], ['+'], ['-']],
    '<ws>': [['<sp1>', '<ws>'], []],
    '<sp1>': [[' '],['\n'],['\t'],['\r']],
    '<symbol>': [[',', '<members>']],
    '<symbol-1>': [[',', '<elements>']],
    '<symbol-2>': [[], ['<symbol>', '<symbol-2>']],
    '<symbol-1-1>': [[], ['<symbol-1>', '<symbol-1-1>']],
    '<character-1>': [[], ['<character>', '<character-1>']],
    '<digit-1>': [['<digit>'], ['<digit>', '<digit-1>']]
}

### An EXPR grammar

In [ ]:
expr_grammar = {
    "<start>": [["<expr>"]],
    "<expr>": [["<term>", "+", "<expr>"], ["<term>", "-", "<expr>"],
               ["<term>"]],
    "<term>": [["<factor>", "*", "<term>"], ["<factor>", "/", "<term>"],
               ["<factor>"]],
    "<factor>": [["+", "<factor>"], ["-", "<factor>"], ["(", "<expr>", ")"],
                 ["<integer>", ".", "<integer>"], ["<integer>"]],
    "<integer>": [["<digit>", "<integer>"], ["<digit>"]],
    "<digit>": [["0"], ["1"], ["2"], ["3"], ["4"], ["5"], ["6"], ["7"], ["8"],
                ["9"]]
}

### An HTML grammar

In [ ]:
# NOTE: HTML grammar requires precomputing key and rule recursion
html_grammar = {
    '<start>': [['<_l_>', '!DOCTYPE html', '<_r_>', '<html_document>']],
    '<_l_>': [['<']],
    '<_r_>': [['>']],
    '<_cl_>': [['</']],
    '<a_tag>':
    [['<_l_>', 'a', '<d>', '<_r_>', '<a_content-1>', '<_cl_>', 'a', '<_r_>']],
    '<a_content>': [['<heading>'], ['<text>']],
    '<abbr_tag>':
    [['<_l_>', 'abbr', '<d>', '<_r_>', '<text>', '<_cl_>', 'abbr', '<_r_>']],
    '<acronym_tag>': [[
        '<_l_>', 'acronym', '<d>', '<_r_>', '<text>', '<_cl_>', 'acronym',
        '<_r_>'
    ]],
    '<address_tag>': [[
        '<_l_>', 'address', '<d>', '<_r_>', '<address_content-1>', '<_cl_>',
        'address', '<_r_>'
    ]],
    '<address_content>': [['<p_tag>'], ['<text>']],
    '<applet_content>': [['<param-1>', '<body_content>']],
    '<area>': [['<_l_>', 'area', '<d>', '<_r_>']],
    '<applet_tag>': [[
        '<_l_>', 'applet', '<d>', '<_r_>', '<applet_content>', '<_cl_>',
        'applet', '<_r_>'
    ]],
    '<b_tag>':
    [['<_l_>', 'b', '<d>', '<_r_>', '<text>', '<_cl_>', 'b', '<_r_>']],
    '<basefont_tag>': [[
        '<_l_>', 'basefront', '<d>', '<_r_>', '<body_content>', '<_cl_>',
        'basefront', '<_r_>'
    ]],
    '<bdo_tag>':
    [['<_l_>', 'bdo', '<d>', '<_r_>', '<text>', '<_cl_>', 'bdo', '<_r_>']],
    '<big_tag>':
    [['<_l_>', 'big', '<d>', '<_r_>', '<text>', '<_cl_>', 'big', '<_r_>']],
    '<blink_tag>':
    [['<_l_>', 'blink', '<d>', '<_r_>', '<text>', '<_cl_>', 'blink', '<_r_>']],
    '<block>': [['<block_content-1>']],
    '<block_content>': [['<basefont_tag>'], ['<blockquote_tag>'],
                        ['<center_tag>'], ['<dir_tag>'], ['<div_tag>'],
                        ['<dl_tag>'], ['<form_tag>'], ['<listing_tag>'],
                        ['<menu_tag>'], ['<multicol_tag>'], ['<nobr_tag>'],
                        ['<ol_tag>'], ['<p_tag>'], ['<pre_tag>'],
                        ['<table_tag>'], ['<ul_tag>'], ['<xmp_tag>']],
    '<blockquote_tag>': [[
        '<_l_>', 'blockquote', '<d>', '<_r_>', '<body_content>', '<_cl_>',
        'blockquote', '<_r_>'
    ]],
    '<body_content>': [['<_l_>', 'bgsound', '<d>', '<_r_>'],
                       ['<_l_>', 'hr', '<_r_>'],
                       ['<address_tag>'], ['<block>'], ['<del_tag>'],
                       ['<heading>'], ['<ins_tag>'], ['<layer_tag>'],
                       ['<map_tag>'], ['<marquee_tag>'], ['<text>']],
    '<body_tag>': [[
        '<_l_>', 'body', '<d>', '<_r_>', '<body_content-1>', '<_cl_>', 'body',
        '<_r_>'
    ]],
    '<caption_tag>': [[
        '<_l_>', 'caption', '<d>', '<_r_>', '<body_content-2>', '<_cl_>',
        'caption', '<_r_>'
    ]],
    '<center_tag>': [[
        '<_l_>', 'center', '<d>', '<_r_>', '<body_content-3>', '<_cl_>',
        'center', '<_r_>'
    ]],
    '<cite_tag>':
    [['<_l_>', 'cite', '<d>', '<_r_>', '<text>', '<_cl_>', 'cite', '<_r_>']],
    '<code_tag>':
    [['<_l_>', 'code', '<d>', '<_r_>', '<text>', '<_cl_>', 'code', '<_r_>']],
    '<colgroup_content>': [['<_l_>', 'col', '<d>', '<_r_-1>']],
    '<colgroup_tag>':
    [['<_l_>', 'colgroup', '<d>', '<_r_>', '<colgroup_content>']],
    '<content_style>': [['<abbr_tag>'], ['<acronym_tag>'], ['<cite_tag>'],
                        ['<code_tag>'], ['<dfn_tag>'], ['<em_tag>'],
                        ['<kbd_tag>'], ['<q_tag>'], ['<strong_tag>'],
                        ['<var_tag>']],
    '<dd_tag>':
    [['<_l_>', 'dd', '<d>', '<_r_>', '<flow>', '<_cl_>', 'dd', '<_r_>']],
    '<del_tag>':
    [['<_l_>', 'del', '<d>', '<_r_>', '<flow>', '<_cl_>', 'del', '<_r_>']],
    '<dfn_tag>':
    [['<_l_>', 'dfn', '<d>', '<_r_>', '<text>', '<_cl_>', 'dfn', '<_r_>']],
    '<dir_tag>': [[
        '<_l_>', 'dir', '<d>', '<_r_>', '<li_tag-1>', '<_cl_>', 'dir', '<_r_>'
    ]],
    '<div_tag>': [[
        '<_l_>', 'div', '<d>', '<_r_>', '<body_content>', '<_cl_>', 'div',
        '<_r_>'
    ]],
    '<dl_content>': [['<dt_tag>', '<dd_tag>']],
    '<dl_tag>': [[
        '<_l_>', 'dl', '<d>', '<_r_>', '<dl_content-1>', '<_cl_>', 'dl',
        '<_r_>'
    ]],
    '<dt_tag>': [[
        '<_l_>', 'dt', '<d>', '<_r_>', '<text>', '<_cl_>', 'dt', '<_r_>'
    ]],
    '<em_tag>': [[
        '<_l_>', 'em', '<d>', '<_r_>', '<text>', '<_cl_>', 'em', '<_r_>'
    ]],
    '<fieldset_tag>': [[
        '<_l_>', 'fieldset', '<d>', '<_r_>', '<legend_tag-1>',
        '<form_content-1>', '<_cl_>', 'fieldset', '<_r_>'
    ]],
    '<flow>': [['<flow_content-1>']],
    '<flow_content>': [['<block>'], ['<text>']],
    '<font_tag>': [[
        '<_l_>', 'font', '<d>', '<_r_>', '<style_text>', '<_cl_>', 'font',
        '<_r_>'
    ]],
    '<form_content>': [['<_l_>', 'input', '<d>', '<_r_>'],
                       ['<_l_>', 'keygen', '<d>', '<_r_>'], ['<body_content>'],
                       ['<fieldset_tag>'], ['<label_tag>'], ['<select_tag>'],
                       ['<textarea_tag>']],
    '<form_tag>': [[
        '<_l_>', 'form', '<d>', '<_r_>', '<form_content-2>', '<_cl_>', 'form',
        '<_r_>'
    ]],
    '<frameset_content>': [['<_l_>', 'frame', '<d>', '<_r_>'],
                           ['<noframes_tag>']],
    '<frameset_tag>': [[
        '<_l_>', 'frameset', '<d>', '<_r_>', '<frameset_content-1>', '<_cl_>',
        'frameset', '<_r_>'
    ]],
    '<h1_tag>': [[
        '<_l_>', 'h1', '<d>', '<_r_>', '<text>', '<_cl_>', 'h1', '<_r_>'
    ]],
    '<h2_tag>': [[
        '<_l_>', 'h2', '<d>', '<_r_>', '<text>', '<_cl_>', 'h2', '<_r_>'
    ]],
    '<h3_tag>': [[
        '<_l_>', 'h3', '<d>', '<_r_>', '<text>', '<_cl_>', 'h3', '<_r_>'
    ]],
    '<h4_tag>': [[
        '<_l_>', 'h4', '<d>', '<_r_>', '<text>', '<_cl_>', 'h4', '<_r_>'
    ]],
    '<h5_tag>': [[
        '<_l_>', 'h5', '<d>', '<_r_>', '<text>', '<_cl_>', 'h5', '<_r_>'
    ]],
    '<h6_tag>': [[
        '<_l_>', 'h6', '<d>', '<_r_>', '<text>', '<_cl_>', 'h6', '<_r_>'
    ]],
    '<head_content>': [['<_l_>', 'base', '<d>', '<_r_>'],
                       ['<_l_>', 'link', '<d>', '<_r_>'],
                       ['<_l_>', 'meta', '<d>', '<_r_>'], ['<style_tag>'],
                       ['<title_tag>'], ['<script_tag>']],
    '<head_tag>': [[
        '<_l_>', 'head', '<d>', '<_r_>', '<head_content-1>', '<_cl_>', 'head',
        '<_r_>'
    ]],
    '<heading>': [['<h1_tag>'], ['<h2_tag>'], ['<h3_tag>'], ['<h4_tag>'],
                  ['<h5_tag>'], ['<h6_tag>']],
    '<html_content>': [['<head_tag>', '<body_tag>'],
                       ['<head_tag>', '<frameset_tag>']],
    '<html_document>': [['<html_tag>']],
    '<html_tag>': [[
        '<_l_>', 'html', '<_r_>', '<html_content>', '<_cl_>', 'html', '<_r_>'
    ]],
    '<i_tag>': [[
        '<_l_>', 'i', '<d>', '<_r_>', '<text>', '<_cl_>', 'i', '<_r_>'
    ]],
    '<ilayer_tag>': [[
        '<_l_>', 'ilayer', '<d>', '<_r_>', '<body_content>', '<_cl_>',
        'ilayer', '<_r_>'
    ]],
    '<ins_tag>': [[
        '<_l_>', 'ins', '<d>', '<_r_>', '<flow>', '<_cl_>', 'ins', '<_r_>'
    ]],
    '<kbd_tag>': [[
        '<_l_>', 'kbd', '<d>', '<_r_>', '<text>', '<_cl_>', 'kbd', '<_r_>'
    ]],
    '<label_content>': [['<_l_>', 'input', '<d>', '<_r_>'], ['<body_content>'],
                        ['<select_tag>'], ['<textarea_tag>']],
    '<label_tag>': [[
        '<_l_>', 'label', '<d>', '<_r_>', '<label_content-1>', '<_cl_>',
        'label', '<_r_>'
    ]],
    '<layer_tag>': [[
        '<_l_>', 'layer', '<d>', '<_r_>', '<body_content>', '<_cl_>', 'layer',
        '<_r_>'
    ]],
    '<legend_tag>': [[
        '<_l_>', 'legend', '<d>', '<_r_>', '<text>', '<_cl_>', 'legend',
        '<_r_>'
    ]],
    '<li_tag>': [[
        '<_l_>', 'li', '<d>', '<_r_>', '<flow>', '<_cl_>', 'li', '<_r_>'
    ]],
    '<literal_text>': [['<plain_text>']],
    '<listing_tag>': [[
        '<_l_>', 'listing', '<d>', '<_r_>', '<literal_text>', '<_cl_>',
        'listing', '<_r_>'
    ]],
    '<map_content>': [['<area-1>']],
    '<map_tag>': [[
        '<_l_>', 'map', '<d>', '<_r_>', '<map_content>', '<_cl_>', 'map',
        '<_r_>'
    ]],
    '<marquee_tag>': [[
        '<_l_>', 'marquee', '<d>', '<_r_>', '<style_text>', '<_cl_>',
        'marquee', '<_r_>'
    ]],
    '<menu_tag>': [[
        '<_l_>', 'menu', '<d>', '<_r_>', '<li_tag-2>', '<_cl_>', 'menu',
        '<_r_>'
    ]],
    '<multicol_tag>': [[
        '<_l_>', 'multicol', '<d>', '<_r_>', '<body_content>', '<_cl_>',
        'multicol', '<_r_>'
    ]],
    '<nobr_tag>': [[
        '<_l_>', 'nobr', '<d>', '<_r_>', '<text>', '<_cl_>', 'nobr', '<_r_>'
    ]],
    '<noembed_tag>': [[
        '<_l_>', 'noembed', '<d>', '<_r_>', '<text>', '<_cl_>', 'noembed',
        '<_r_>'
    ]],
    '<noframes_tag>': [[
        '<_l_>', 'noframes', '<d>', '<_r_>', '<body_content-4>', '<_cl_>',
        'noframes', '<_r_>'
    ]],
    '<noscript_tag>': [[
        '<_l_>', 'noscript', '<d>', '<_r_>', '<text>', '<_cl_>', 'noscript',
        '<_r_>'
    ]],
    '<object_content>': [['<param-2>', '<body_content>']],
    '<object_tag>': [[
        '<_l_>', 'object', '<d>', '<_r_>', '<object_content>', '<_cl_>',
        'object', '<_r_>'
    ]],
    '<ol_tag>': [[
        '<_l_>', 'ol', '<d>', '<_r_>', '<li_tag-3>', '<_cl_>', 'ol', '<_r_>'
    ]],
    '<optgroup_tag>': [[
        '<_l_>', 'optgroup', '<d>', '<_r_>', '<option_tag-1>', '<_cl_>',
        'optgroup', '<_r_>'
    ]],
    '<option_tag>': [[
        '<_l_>', 'option', '<d>', '<_r_>', '<plain_text-1>', '<_cl_>',
        'option', '<_r_>'
    ]],
    '<p_tag>': [['<_l_>', 'p', '<_r_>', '<text>', '<_cl_>', 'p', '<_r_>']],
    '<param>': [['<_l_>', 'param', '<_r_>']],
    '<plain_text>': [['<entity-1>']],
    '<entity>': [['<char>'], ['<ampersand>']],
    '<char>': [['7'], ['*'], [':'], [']'], ['n'], ['m'], ['N'], ['/'], ['.'],
               ['K'], ['T'], ['I'], ['f'], ['o'], [','], ['l'], ['W'], ['-'],
               ['?'], ['\\'], ['%'], ['1'], ['c'], ['H'], ['!'], ['A'], ['$'],
               ['9'], ['q'], ['['], [')'], [' '], [';'], ['b'], ['i'], ['L'],
               ["'"], ['Y'], ['\t'], ['3'], ['g'], ['F'], ['E'], ['D'], ['C'],
               ['@'], ['t'], ['R'], ['"'], ['2'], ['}'], ['~'], ['5'], ['4'],
               ['z'], ['X'], ['S'], ['O'], ['v'], ['J'], ['`'], ['B'], ['\n'],
               ['y'], ['p'], ['6'], ['0'], ['k'], ['w'], ['\r'], ['V'], ['_'],
               ['s'], ['x'], ['{'], ['d'], ['a'], ['#'], ['Q'], ['<'], ['u'],
               ['r'], ['U'], ['h'], ['>'], ['('], ['P'], ['G'], ['\x0c'],
               ['Z'], ['j'], ['|'], ['e'], ['^'], ['='], ['8'], ['+'], ['M']],
    '<ampersand>': [['&nbsp;']],
    '<physical_style>': [['<b_tag>'], ['<bdo_tag>'], ['<big_tag>'],
                         ['<blink_tag>'], ['<font_tag>'], ['<i_tag>'],
                         ['<s_tag>'], ['<small_tag>'], ['<span_tag>'],
                         ['<strike_tag>'], ['<sub_tag>'], ['<sup_tag>'],
                         ['<tt_tag>'], ['<u_tag>']],
    '<pre_content>': [['<_l_>', 'br', '<_r_>'], ['<_l_>', 'hr', '<_r_>'],
                      ['<a_tag>'], ['<style_text>']],
    '<pre_tag>': [[
        '<_l_>', 'pre', '<_r_>', '<pre_content-1>', '<_cl_>', 'pre', '<_r_>'
    ]],
    '<q_tag>': [['<_l_>', 'q', '<_r_>', '<text>', '<_cl_>', 'q', '<_r_>']],
    '<s_tag>': [['<_l_>', 's', '<_r_>', '<text>', '<_cl_>', 's', '<_r_>']],
    '<script_tag>': [[
        '<_l_>', 'script', '<d>', '<_r_>', '<plain_text>', '<_cl_>', 'script',
        '<_r_>'
    ]],
    '<select_content>': [['<optgroup_tag>'], ['<option_tag>']],
    '<select_tag>': [[
        '<_l_>', 'select', '<d>', '<_r_>', '<select_content-1>', '<_cl_>',
        'select', '<_r_>'
    ]],
    '<small_tag>': [[
        '<_l_>', 'small', '<d>', '<_r_>', '<text>', '<_cl_>', 'small', '<_r_>'
    ]],
    '<span_tag>': [[
        '<_l_>', 'span', '<d>', '<_r_>', '<text>', '<_cl_>', 'span', '<_r_>'
    ]],
    '<strike_tag>': [[
        '<_l_>', 'strike', '<d>', '<_r_>', '<text>', '<_cl_>', 'strike',
        '<_r_>'
    ]],
    '<strong_tag>': [[
        '<_l_>', 'strong', '<d>', '<_r_>', '<text>', '<_cl_>', 'strong',
        '<_r_>'
    ]],
    '<style_tag>': [[
        '<_l_>', 'style', '<d>', '<_r_>', '<plain_text>', '<_cl_>', 'style',
        '<_r_>'
    ]],
    '<style_text>': [['<plain_text>']],
    '<sub_tag>': [[
        '<_l_>', 'sub', '<d>', '<_r_>', '<text>', '<_cl_>', 'sub', '<_r_>'
    ]],
    '<sup_tag>': [[
        '<_l_>', 'sup', '<d>', '<_r_>', '<text>', '<_cl_>', 'sup', '<_r_>'
    ]],
    '<table_cell>': [['<td_tag>'], ['<th_tag>']],
    '<table_content>': [['<_l_>', 'tbody', '<d>', '<_r_>'],
                        ['<_l_>', 'tfoot', '<d>', '<_r_>'],
                        ['<_l_>', 'thead', '<d>', '<_r_>'], ['<tr_tag>']],
    '<table_tag>': [[
        '<_l_>', 'table', '<d>', '<_r_>', '<caption_tag-1>',
        '<colgroup_tag-1>', '<table_content-1>', '<_cl_>', 'table', '<_r_>'
    ]],
    '<td_tag>': [[
        '<_l_>', 'td', '<d>', '<_r_>', '<body_content>', '<_cl_>', 'td',
        '<_r_>'
    ]],
    '<text>': [['<text_content-1>']],
    '<text_content>': [['<_l_>', 'br', '<d>', '<_r_>'],
                       ['<_l_>', 'embed', '<d>', '<_r_>'],
                       ['<_l_>', 'iframe', '<d>', '<_r_>'],
                       ['<_l_>', 'img', '<d>', '<_r_>'],
                       ['<_l_>', 'spacer', '<d>', '<_r_>'],
                       ['<_l_>', 'wbr', '<d>', '<_r_>'], ['<a_tag>'],
                       ['<applet_tag>'], ['<content_style>'], ['<ilayer_tag>'],
                       ['<noembed_tag>'], ['<noscript_tag>'], ['<object_tag>'],
                       ['<plain_text>'], ['<physical_style>']],
    '<textarea_tag>': [[
        '<_l_>', 'textarea', '<d>', '<_r_>', '<plain_text>', '<_cl_>',
        'textarea', '<_r_>'
    ]],
    '<th_tag>': [[
        '<_l_>', 'th', '<d>', '<_r_>', '<body_content>', '<_cl_>', 'th',
        '<_r_>'
    ]],
    '<title_tag>': [[
        '<_l_>', 'title', '<d>', '<_r_>', '<plain_text>', '<_cl_>', 'title',
        '<_r_>'
    ]],
    '<tr_tag>': [[
        '<_l_>', 'tr', '<d>', '<_r_>', '<table_cell-1>', '<_cl_>', 'tr',
        '<_r_>'
    ]],
    '<tt_tag>': [[
        '<_l_>', 'tt', '<d>', '<_r_>', '<text>', '<_cl_>', 'tt', '<_r_>'
    ]],
    '<u_tag>': [[
        '<_l_>', 'u', '<d>', '<_r_>', '<text>', '<_cl_>', 'u', '<_r_>'
    ]],
    '<ul_tag>': [[
        '<_l_>', 'ul', '<d>', '<_r_>', '<li_tag-4>', '<_cl_>', 'ul', '<_r_>'
    ]],
    '<var_tag>': [[
        '<_l_>', 'var', '<d>', '<_r_>', '<text>', '<_cl_>', 'var', '<_r_>'
    ]],
    '<xmp_tag>': [[
        '<_l_>', 'xmp', '<d>', '<_r_>', '<literal_text>', '<_cl_>', 'xmp',
        '<_r_>'
    ]],
    '<d>': [['<space-1>', '<attributes-1>', '<space-2>'], []],
    '<attribute>': [['<key>'], ['<key>', '="', '<value>', '"'],
                    ['<key>', "='", '<value>', "'"],
                    ['<key>', '=', '<uqvalue>']],
    '<key>': [['<allchars>']],
    '<allchars>': [
        ['7'], ['*'], [':'], ['&'], [']'], ['n'], ['m'], ['N'], ['.'], ['K'],
        ['T'], ['I'], ['f'], ['o'], [','], ['l'], ['W'], ['-'], ['?'], ['\\'],
        ['%'], ['1'], ['c'], ['H'], ['!'], ['A'], ['$'], ['9'], ['q'], ['['],
        [')'], [';'], ['b'], ['i'], ['L'], ['Y'], ['3'], ['g'], ['F'], ['E'],
        ['D'], ['C'], ['@'], ['t'], ['R'], ['2'], ['}'], ['~'], ['5'], ['4'],
        ['z'], ['X'], ['S'], ['O'], ['v'], ['J'], ['`'], ['B'], ['y'], ['p'],
        ['6'], ['0'], ['k'], ['w'], ['\r'], ['V'], ['_'], ['s'], ['x'], ['{'],
        ['d'], ['a'], ['#'], ['Q'], ['u'], ['r'], ['U'], ['h'], ['('], ['P'],
        ['G'], ['\x0c'], ['Z'], ['j'], ['|'], ['e'], ['^'], ['8'], ['+'],
        ['M']
    ],
    '<value>': [['<anychars>']],
    '<anychar>': [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'],
                  ['8'], ['9'], ['a'], ['b'], ['c'], ['d'], ['e'], ['f'
                                                                    ], ['g'],
                  ['h'], ['i'], ['j'], ['k'], ['l'], ['m'], ['n'], ['o'
                                                                    ], ['p'],
                  ['q'], ['r'], ['s'], ['t'], ['u'], ['v'], ['w'], ['x'
                                                                    ], ['y'],
                  ['z'], ['A'], ['B'], ['C'], ['D'], ['E'], ['F'], ['G'
                                                                    ], ['H'],
                  ['I'], ['J'], ['K'], ['L'], ['M'], ['N'], ['O'], ['P'
                                                                    ], ['Q'],
                  ['R'], ['S'], ['T'], ['U'], ['V'], ['W'], ['X'], ['Y'
                                                                    ], ['Z'],
                  ['!'], ['"'], ['#'], ['$'], ['%'], ['&'], ["'"], ['('
                                                                    ], [')'],
                  ['*'], ['+'], [','], ['-'], ['.'], ['/'], [':'], [';'
                                                                    ], ['<'],
                  ['='], ['>'], ['?'], ['@'], ['['], ['\\'], [']'], ['^'],
                  ['_'], ['`'], ['{'], ['|'], ['}'], ['~'], [' '], ['\t'],
                  ['\n'], ['\r'], ['\x0b'], ['\x0c']],
    '<anychars>': [['<anychar-1>']],
    '<uqvalue>': [['<uqchars>']],
    '<uqchar>': [['7'], ['*'], [':'], ['&'], [']'], ['n'], ['m'], ['N'], ['.'],
                 ['K'], ['T'], ['I'], ['f'], ['o'], [','], ['l'], ['W'], ['-'],
                 ['?'], ['\\'], ['%'], ['1'], ['c'], ['H'], ['!'], ['A'],
                 ['$'], ['9'], ['q'], ['['], [')'], [';'], ['b'], ['i'], ['L'],
                 ['Y'], ['3'], ['g'], ['F'], ['E'], ['D'], ['C'], ['@'], ['t'],
                 ['R'], ['2'], ['}'], ['~'], ['5'], ['4'], ['z'], ['X'], ['S'],
                 ['O'], ['v'], ['J'], ['B'], ['y'], ['p'], ['6'], ['0'], ['k'],
                 ['w'], ['\r'], ['V'], ['_'], ['s'], ['x'], ['{'], ['d'],
                 ['a'], ['#'], ['Q'], ['u'], ['r'], ['U'], ['h'], ['('], ['P'],
                 ['G'], ['\x0c'], ['Z'], ['j'], ['|'], ['e'], ['^'], ['8'],
                 ['+'], ['M']],
    '<uqchars>': [['<uqchar-1>']],
    '<attributes>': [['<attribute>'],
                     ['<attribute>', '<space-3>', '<attributes>']],
    '<space>': [[' '], ['\t'], ['\n']],
    '<a_content-1>': [[], ['<a_content>', '<a_content-1>']],
    '<address_content-1>': [[], ['<address_content>', '<address_content-1>']],
    '<param-1>': [[], ['<param>', '<param-1>']],
    '<block_content-1>': [[], ['<block_content>', '<block_content-1>']],
    '<body_content-1>': [[], ['<body_content>', '<body_content-1>']],
    '<body_content-2>': [[], ['<body_content>', '<body_content-2>']],
    '<body_content-3>': [[], ['<body_content>', '<body_content-3>']],
    '<_r_-1>': [[], ['<_r_>', '<_r_-1>']],
    '<li_tag-1>': [['<li_tag>'], ['<li_tag>', '<li_tag-1>']],
    '<dl_content-1>': [['<dl_content>'], ['<dl_content>', '<dl_content-1>']],
    '<legend_tag-1>': [[], ['<legend_tag>', '<legend_tag-1>']],
    '<form_content-1>': [[], ['<form_content>', '<form_content-1>']],
    '<flow_content-1>': [[], ['<flow_content>', '<flow_content-1>']],
    '<form_content-2>': [[], ['<form_content>', '<form_content-2>']],
    '<frameset_content-1>': [[],
                             ['<frameset_content>', '<frameset_content-1>']],
    '<head_content-1>': [[], ['<head_content>', '<head_content-1>']],
    '<label_content-1>': [[], ['<label_content>', '<label_content-1>']],
    '<area-1>': [[], ['<area>', '<area-1>']],
    '<li_tag-2>': [[], ['<li_tag>', '<li_tag-2>']],
    '<body_content-4>': [[], ['<body_content>', '<body_content-4>']],
    '<param-2>': [[], ['<param>', '<param-2>']],
    '<li_tag-3>': [['<li_tag>'], ['<li_tag>', '<li_tag-3>']],
    '<option_tag-1>': [[], ['<option_tag>', '<option_tag-1>']],
    '<plain_text-1>': [['<plain_text>'], ['<plain_text>', '<plain_text-1>']],
    '<entity-1>': [[], ['<entity>', '<entity-1>']],
    '<pre_content-1>': [[], ['<pre_content>', '<pre_content-1>']],
    '<select_content-1>': [[], ['<select_content>', '<select_content-1>']],
    '<caption_tag-1>': [[], ['<caption_tag>', '<caption_tag-1>']],
    '<colgroup_tag-1>': [[], ['<colgroup_tag>', '<colgroup_tag-1>']],
    '<table_content-1>': [[], ['<table_content>', '<table_content-1>']],
    '<text_content-1>': [[], ['<text_content>', '<text_content-1>']],
    '<table_cell-1>': [[], ['<table_cell>', '<table_cell-1>']],
    '<li_tag-4>': [[], ['<li_tag>', '<li_tag-4>']],
    '<space-1>': [['<space>'], ['<space>', '<space-1>']],
    '<attributes-1>': [[], ['<attributes>', '<attributes-1>']],
    '<space-2>': [[], ['<space>', '<space-2>']],
    '<anychar-1>': [[], ['<anychar>', '<anychar-1>']],
    '<uqchar-1>': [['<uqchar>'], ['<uqchar>', '<uqchar-1>']],
    
    '<space-3>': [['<space>'], ['<space>', '<space-3>']]
}

In [ ]:
# Warning -- set to False if not HTML
IS_HTML = False

In [ ]:
HTML_KEY_RECURSION = {'<start>': False,
 '<_l_>': False,
 '<_r_>': False,
 '<_cl_>': False,
 '<a_tag>': False,
 '<a_content>': False,
 '<abbr_tag>': False,
 '<acronym_tag>': False,
 '<address_tag>': False,
 '<address_content>': False,
 '<applet_content>': False,
 '<area>': False,
 '<applet_tag>': False,
 '<b_tag>': False,
 '<basefont_tag>': False,
 '<bdo_tag>': False,
 '<big_tag>': False,
 '<blink_tag>': False,
 '<block>': False,
 '<block_content>': False,
 '<blockquote_tag>': False,
 '<body_content>': False,
 '<body_tag>': False,
 '<caption_tag>': False,
 '<center_tag>': False,
 '<cite_tag>': False,
 '<code_tag>': False,
 '<colgroup_content>': False,
 '<colgroup_tag>': False,
 '<content_style>': False,
 '<dd_tag>': False,
 '<del_tag>': False,
 '<dfn_tag>': False,
 '<dir_tag>': False,
 '<div_tag>': False,
 '<dl_content>': False,
 '<dl_tag>': False,
 '<dt_tag>': False,
 '<em_tag>': False,
 '<fieldset_tag>': False,
 '<flow>': False,
 '<flow_content>': False,
 '<font_tag>': False,
 '<form_content>': False,
 '<form_tag>': False,
 '<frameset_content>': False,
 '<frameset_tag>': False,
 '<h1_tag>': False,
 '<h2_tag>': False,
 '<h3_tag>': False,
 '<h4_tag>': False,
 '<h5_tag>': False,
 '<h6_tag>': False,
 '<head_content>': False,
 '<head_tag>': False,
 '<heading>': False,
 '<html_content>': False,
 '<html_document>': False,
 '<html_tag>': False,
 '<i_tag>': False,
 '<ilayer_tag>': False,
 '<ins_tag>': False,
 '<kbd_tag>': False,
 '<label_content>': False,
 '<label_tag>': False,
 '<layer_tag>': False,
 '<legend_tag>': False,
 '<li_tag>': False,
 '<literal_text>': False,
 '<listing_tag>': False,
 '<map_content>': False,
 '<map_tag>': False,
 '<marquee_tag>': False,
 '<menu_tag>': False,
 '<multicol_tag>': False,
 '<nobr_tag>': False,
 '<noembed_tag>': False,
 '<noframes_tag>': False,
 '<noscript_tag>': False,
 '<object_content>': False,
 '<object_tag>': False,
 '<ol_tag>': False,
 '<optgroup_tag>': False,
 '<option_tag>': False,
 '<p_tag>': False,
 '<param>': False,
 '<plain_text>': False,
 '<entity>': False,
 '<char>': False,
 '<ampersand>': False,
 '<physical_style>': False,
 '<pre_content>': False,
 '<pre_tag>': False,
 '<q_tag>': False,
 '<s_tag>': False,
 '<script_tag>': False,
 '<select_content>': False,
 '<select_tag>': False,
 '<small_tag>': False,
 '<span_tag>': False,
 '<strike_tag>': False,
 '<strong_tag>': False,
 '<style_tag>': False,
 '<style_text>': False,
 '<sub_tag>': False,
 '<sup_tag>': False,
 '<table_cell>': False,
 '<table_content>': False,
 '<table_tag>': False,
 '<td_tag>': False,
 '<text>': False,
 '<text_content>': False,
 '<textarea_tag>': False,
 '<th_tag>': False,
 '<title_tag>': False,
 '<tr_tag>': False,
 '<tt_tag>': False,
 '<u_tag>': False,
 '<ul_tag>': False,
 '<var_tag>': False,
 '<xmp_tag>': False,
 '<d>': False,
 '<attribute>': False,
 '<key>': False,
 '<allchars>': False,
 '<value>': False,
 '<anychar>': False,
 '<anychars>': False,
 '<uqvalue>': False,
 '<uqchar>': False,
 '<uqchars>': False,
 '<attributes>': True,
 '<space>': False,
 '<a_content_1>': True,
 '<address_content_1>': True,
 '<param_1>': True,
 '<block_content_1>': True,
 '<body_content_1>': True,
 '<body_content_2>': True,
 '<body_content_3>': True,
 '<_r__1>': True,
 '<li_tag_1>': True,
 '<dl_content_1>': True,
 '<legend_tag_1>': True,
 '<form_content_1>': True,
 '<flow_content_1>': True,
 '<form_content_2>': True,
 '<frameset_content_1>': True,
 '<head_content_1>': True,
 '<label_content_1>': True,
 '<area_1>': True,
 '<li_tag_2>': True,
 '<body_content_4>': True,
 '<param_2>': True,
 '<li_tag_3>': True,
 '<option_tag_1>': True,
 '<plain_text_1>': True,
 '<entity_1>': True,
 '<pre_content_1>': True,
 '<select_content_1>': True,
 '<caption_tag_1>': True,
 '<colgroup_tag_1>': True,
 '<table_content_1>': True,
 '<text_content_1>': True,
 '<table_cell_1>': True,
 '<li_tag_4>': True,
 '<space_1>': True,
 '<attributes_1>': True,
 '<space_2>': True,
 '<anychar_1>': True,
 '<uqchar_1>': True,
 '<space_3>': True}

HTML_RULE_RECURSION = {'gen_start_0': False,
 'gen__l__0': False,
 'gen__r__0': False,
 'gen__cl__0': False,
 'gen_a_tag_0': True,
 'gen_a_content_0': True,
 'gen_a_content_1': True,
 'gen_abbr_tag_0': True,
 'gen_acronym_tag_0': True,
 'gen_address_tag_0': True,
 'gen_address_content_0': True,
 'gen_address_content_1': True,
 'gen_applet_content_0': True,
 'gen_area_0': False,
 'gen_applet_tag_0': True,
 'gen_b_tag_0': True,
 'gen_basefont_tag_0': True,
 'gen_bdo_tag_0': True,
 'gen_big_tag_0': True,
 'gen_blink_tag_0': True,
 'gen_block_0': True,
 'gen_block_content_0': True,
 'gen_block_content_1': True,
 'gen_block_content_2': True,
 'gen_block_content_3': True,
 'gen_block_content_4': True,
 'gen_block_content_5': True,
 'gen_block_content_6': True,
 'gen_block_content_7': True,
 'gen_block_content_8': False,
 'gen_block_content_9': False,
 'gen_block_content_10': True,
 'gen_block_content_11': True,
 'gen_block_content_12': True,
 'gen_block_content_13': True,
 'gen_block_content_14': True,
 'gen_block_content_15': True,
 'gen_block_content_16': True,
 'gen_blockquote_tag_0': True,
 'gen_body_content_0': False,
 'gen_body_content_1': False,
 'gen_body_content_2': True,
 'gen_body_content_3': True,
 'gen_body_content_4': True,
 'gen_body_content_5': True,
 'gen_body_content_6': True,
 'gen_body_content_7': False,
 'gen_body_content_8': False,
 'gen_body_content_9': True,
 'gen_body_content_10': True,
 'gen_body_tag_0': False,
 'gen_caption_tag_0': True,
 'gen_center_tag_0': True,
 'gen_cite_tag_0': True,
 'gen_code_tag_0': True,
 'gen_colgroup_content_0': False,
 'gen_colgroup_tag_0': False,
 'gen_content_style_0': True,
 'gen_content_style_1': True,
 'gen_content_style_2': True,
 'gen_content_style_3': True,
 'gen_content_style_4': True,
 'gen_content_style_5': True,
 'gen_content_style_6': True,
 'gen_content_style_7': True,
 'gen_content_style_8': True,
 'gen_content_style_9': True,
 'gen_dd_tag_0': True,
 'gen_del_tag_0': True,
 'gen_dfn_tag_0': True,
 'gen_dir_tag_0': True,
 'gen_div_tag_0': True,
 'gen_dl_content_0': True,
 'gen_dl_tag_0': True,
 'gen_dt_tag_0': True,
 'gen_em_tag_0': True,
 'gen_fieldset_tag_0': True,
 'gen_flow_0': True,
 'gen_flow_content_0': True,
 'gen_flow_content_1': True,
 'gen_font_tag_0': False,
 'gen_form_content_0': False,
 'gen_form_content_1': False,
 'gen_form_content_2': True,
 'gen_form_content_3': True,
 'gen_form_content_4': True,
 'gen_form_content_5': False,
 'gen_form_content_6': False,
 'gen_form_tag_0': True,
 'gen_frameset_content_0': False,
 'gen_frameset_content_1': False,
 'gen_frameset_tag_0': False,
 'gen_h1_tag_0': True,
 'gen_h2_tag_0': True,
 'gen_h3_tag_0': True,
 'gen_h4_tag_0': True,
 'gen_h5_tag_0': True,
 'gen_h6_tag_0': True,
 'gen_head_content_0': False,
 'gen_head_content_1': False,
 'gen_head_content_2': False,
 'gen_head_content_3': False,
 'gen_head_content_4': False,
 'gen_head_content_5': False,
 'gen_head_tag_0': False,
 'gen_heading_0': True,
 'gen_heading_1': True,
 'gen_heading_2': True,
 'gen_heading_3': True,
 'gen_heading_4': True,
 'gen_heading_5': True,
 'gen_html_content_0': False,
 'gen_html_content_1': False,
 'gen_html_document_0': False,
 'gen_html_tag_0': False,
 'gen_i_tag_0': True,
 'gen_ilayer_tag_0': True,
 'gen_ins_tag_0': True,
 'gen_kbd_tag_0': True,
 'gen_label_content_0': False,
 'gen_label_content_1': True,
 'gen_label_content_2': False,
 'gen_label_content_3': False,
 'gen_label_tag_0': True,
 'gen_layer_tag_0': True,
 'gen_legend_tag_0': True,
 'gen_li_tag_0': True,
 'gen_literal_text_0': False,
 'gen_listing_tag_0': False,
 'gen_map_content_0': False,
 'gen_map_tag_0': False,
 'gen_marquee_tag_0': False,
 'gen_menu_tag_0': True,
 'gen_multicol_tag_0': True,
 'gen_nobr_tag_0': True,
 'gen_noembed_tag_0': True,
 'gen_noframes_tag_0': False,
 'gen_noscript_tag_0': True,
 'gen_object_content_0': True,
 'gen_object_tag_0': True,
 'gen_ol_tag_0': True,
 'gen_optgroup_tag_0': False,
 'gen_option_tag_0': False,
 'gen_p_tag_0': True,
 'gen_param_0': False,
 'gen_plain_text_0': False,
 'gen_entity_0': False,
 'gen_entity_1': False,
 'gen_char_0': False,
 'gen_char_1': False,
 'gen_char_2': False,
 'gen_char_3': False,
 'gen_char_4': False,
 'gen_char_5': False,
 'gen_char_6': False,
 'gen_char_7': False,
 'gen_char_8': False,
 'gen_char_9': False,
 'gen_char_10': False,
 'gen_char_11': False,
 'gen_char_12': False,
 'gen_char_13': False,
 'gen_char_14': False,
 'gen_char_15': False,
 'gen_char_16': False,
 'gen_char_17': False,
 'gen_char_18': False,
 'gen_char_19': False,
 'gen_char_20': False,
 'gen_char_21': False,
 'gen_char_22': False,
 'gen_char_23': False,
 'gen_char_24': False,
 'gen_char_25': False,
 'gen_char_26': False,
 'gen_char_27': False,
 'gen_char_28': False,
 'gen_char_29': False,
 'gen_char_30': False,
 'gen_char_31': False,
 'gen_char_32': False,
 'gen_char_33': False,
 'gen_char_34': False,
 'gen_char_35': False,
 'gen_char_36': False,
 'gen_char_37': False,
 'gen_char_38': False,
 'gen_char_39': False,
 'gen_char_40': False,
 'gen_char_41': False,
 'gen_char_42': False,
 'gen_char_43': False,
 'gen_char_44': False,
 'gen_char_45': False,
 'gen_char_46': False,
 'gen_char_47': False,
 'gen_char_48': False,
 'gen_char_49': False,
 'gen_char_50': False,
 'gen_char_51': False,
 'gen_char_52': False,
 'gen_char_53': False,
 'gen_char_54': False,
 'gen_char_55': False,
 'gen_char_56': False,
 'gen_char_57': False,
 'gen_char_58': False,
 'gen_char_59': False,
 'gen_char_60': False,
 'gen_char_61': False,
 'gen_char_62': False,
 'gen_char_63': False,
 'gen_char_64': False,
 'gen_char_65': False,
 'gen_char_66': False,
 'gen_char_67': False,
 'gen_char_68': False,
 'gen_char_69': False,
 'gen_char_70': False,
 'gen_char_71': False,
 'gen_char_72': False,
 'gen_char_73': False,
 'gen_char_74': False,
 'gen_char_75': False,
 'gen_char_76': False,
 'gen_char_77': False,
 'gen_char_78': False,
 'gen_char_79': False,
 'gen_char_80': False,
 'gen_char_81': False,
 'gen_char_82': False,
 'gen_char_83': False,
 'gen_char_84': False,
 'gen_char_85': False,
 'gen_char_86': False,
 'gen_char_87': False,
 'gen_char_88': False,
 'gen_char_89': False,
 'gen_char_90': False,
 'gen_char_91': False,
 'gen_char_92': False,
 'gen_char_93': False,
 'gen_char_94': False,
 'gen_char_95': False,
 'gen_char_96': False,
 'gen_char_97': False,
 'gen_ampersand_0': False,
 'gen_physical_style_0': False,
 'gen_physical_style_1': True,
 'gen_physical_style_2': True,
 'gen_physical_style_3': True,
 'gen_physical_style_4': True,
 'gen_physical_style_5': True,
 'gen_physical_style_6': True,
 'gen_physical_style_7': True,
 'gen_physical_style_8': True,
 'gen_physical_style_9': True,
 'gen_physical_style_10': True,
 'gen_physical_style_11': True,
 'gen_physical_style_12': True,
 'gen_physical_style_13': True,
 'gen_pre_content_0': False,
 'gen_pre_content_1': False,
 'gen_pre_content_2': True,
 'gen_pre_content_3': False,
 'gen_pre_tag_0': True,
 'gen_q_tag_0': True,
 'gen_s_tag_0': True,
 'gen_script_tag_0': False,
 'gen_select_content_0': False,
 'gen_select_content_1': False,
 'gen_select_tag_0': False,
 'gen_small_tag_0': True,
 'gen_span_tag_0': True,
 'gen_strike_tag_0': True,
 'gen_strong_tag_0': True,
 'gen_style_tag_0': False,
 'gen_style_text_0': False,
 'gen_sub_tag_0': True,
 'gen_sup_tag_0': True,
 'gen_table_cell_0': True,
 'gen_table_cell_1': True,
 'gen_table_content_0': False,
 'gen_table_content_1': False,
 'gen_table_content_2': False,
 'gen_table_content_3': True,
 'gen_table_tag_0': True,
 'gen_td_tag_0': True,
 'gen_text_0': True,
 'gen_text_content_0': False,
 'gen_text_content_1': False,
 'gen_text_content_2': False,
 'gen_text_content_3': False,
 'gen_text_content_4': False,
 'gen_text_content_5': False,
 'gen_text_content_6': True,
 'gen_text_content_7': False,
 'gen_text_content_8': True,
 'gen_text_content_9': True,
 'gen_text_content_10': True,
 'gen_text_content_11': True,
 'gen_text_content_12': True,
 'gen_text_content_13': True,
 'gen_text_content_14': True,
 'gen_textarea_tag_0': False,
 'gen_th_tag_0': True,
 'gen_title_tag_0': False,
 'gen_tr_tag_0': True,
 'gen_tt_tag_0': True,
 'gen_u_tag_0': True,
 'gen_ul_tag_0': True,
 'gen_var_tag_0': True,
 'gen_xmp_tag_0': False,
 'gen_d_0': False,
 'gen_d_1': False,
 'gen_attribute_0': False,
 'gen_attribute_1': False,
 'gen_attribute_2': False,
 'gen_attribute_3': False,
 'gen_key_0': False,
 'gen_allchars_0': False,
 'gen_allchars_1': False,
 'gen_allchars_2': False,
 'gen_allchars_3': False,
 'gen_allchars_4': False,
 'gen_allchars_5': False,
 'gen_allchars_6': False,
 'gen_allchars_7': False,
 'gen_allchars_8': False,
 'gen_allchars_9': False,
 'gen_allchars_10': False,
 'gen_allchars_11': False,
 'gen_allchars_12': False,
 'gen_allchars_13': False,
 'gen_allchars_14': False,
 'gen_allchars_15': False,
 'gen_allchars_16': False,
 'gen_allchars_17': False,
 'gen_allchars_18': False,
 'gen_allchars_19': False,
 'gen_allchars_20': False,
 'gen_allchars_21': False,
 'gen_allchars_22': False,
 'gen_allchars_23': False,
 'gen_allchars_24': False,
 'gen_allchars_25': False,
 'gen_allchars_26': False,
 'gen_allchars_27': False,
 'gen_allchars_28': False,
 'gen_allchars_29': False,
 'gen_allchars_30': False,
 'gen_allchars_31': False,
 'gen_allchars_32': False,
 'gen_allchars_33': False,
 'gen_allchars_34': False,
 'gen_allchars_35': False,
 'gen_allchars_36': False,
 'gen_allchars_37': False,
 'gen_allchars_38': False,
 'gen_allchars_39': False,
 'gen_allchars_40': False,
 'gen_allchars_41': False,
 'gen_allchars_42': False,
 'gen_allchars_43': False,
 'gen_allchars_44': False,
 'gen_allchars_45': False,
 'gen_allchars_46': False,
 'gen_allchars_47': False,
 'gen_allchars_48': False,
 'gen_allchars_49': False,
 'gen_allchars_50': False,
 'gen_allchars_51': False,
 'gen_allchars_52': False,
 'gen_allchars_53': False,
 'gen_allchars_54': False,
 'gen_allchars_55': False,
 'gen_allchars_56': False,
 'gen_allchars_57': False,
 'gen_allchars_58': False,
 'gen_allchars_59': False,
 'gen_allchars_60': False,
 'gen_allchars_61': False,
 'gen_allchars_62': False,
 'gen_allchars_63': False,
 'gen_allchars_64': False,
 'gen_allchars_65': False,
 'gen_allchars_66': False,
 'gen_allchars_67': False,
 'gen_allchars_68': False,
 'gen_allchars_69': False,
 'gen_allchars_70': False,
 'gen_allchars_71': False,
 'gen_allchars_72': False,
 'gen_allchars_73': False,
 'gen_allchars_74': False,
 'gen_allchars_75': False,
 'gen_allchars_76': False,
 'gen_allchars_77': False,
 'gen_allchars_78': False,
 'gen_allchars_79': False,
 'gen_allchars_80': False,
 'gen_allchars_81': False,
 'gen_allchars_82': False,
 'gen_allchars_83': False,
 'gen_allchars_84': False,
 'gen_allchars_85': False,
 'gen_allchars_86': False,
 'gen_allchars_87': False,
 'gen_allchars_88': False,
 'gen_allchars_89': False,
 'gen_value_0': False,
 'gen_anychar_0': False,
 'gen_anychar_1': False,
 'gen_anychar_2': False,
 'gen_anychar_3': False,
 'gen_anychar_4': False,
 'gen_anychar_5': False,
 'gen_anychar_6': False,
 'gen_anychar_7': False,
 'gen_anychar_8': False,
 'gen_anychar_9': False,
 'gen_anychar_10': False,
 'gen_anychar_11': False,
 'gen_anychar_12': False,
 'gen_anychar_13': False,
 'gen_anychar_14': False,
 'gen_anychar_15': False,
 'gen_anychar_16': False,
 'gen_anychar_17': False,
 'gen_anychar_18': False,
 'gen_anychar_19': False,
 'gen_anychar_20': False,
 'gen_anychar_21': False,
 'gen_anychar_22': False,
 'gen_anychar_23': False,
 'gen_anychar_24': False,
 'gen_anychar_25': False,
 'gen_anychar_26': False,
 'gen_anychar_27': False,
 'gen_anychar_28': False,
 'gen_anychar_29': False,
 'gen_anychar_30': False,
 'gen_anychar_31': False,
 'gen_anychar_32': False,
 'gen_anychar_33': False,
 'gen_anychar_34': False,
 'gen_anychar_35': False,
 'gen_anychar_36': False,
 'gen_anychar_37': False,
 'gen_anychar_38': False,
 'gen_anychar_39': False,
 'gen_anychar_40': False,
 'gen_anychar_41': False,
 'gen_anychar_42': False,
 'gen_anychar_43': False,
 'gen_anychar_44': False,
 'gen_anychar_45': False,
 'gen_anychar_46': False,
 'gen_anychar_47': False,
 'gen_anychar_48': False,
 'gen_anychar_49': False,
 'gen_anychar_50': False,
 'gen_anychar_51': False,
 'gen_anychar_52': False,
 'gen_anychar_53': False,
 'gen_anychar_54': False,
 'gen_anychar_55': False,
 'gen_anychar_56': False,
 'gen_anychar_57': False,
 'gen_anychar_58': False,
 'gen_anychar_59': False,
 'gen_anychar_60': False,
 'gen_anychar_61': False,
 'gen_anychar_62': False,
 'gen_anychar_63': False,
 'gen_anychar_64': False,
 'gen_anychar_65': False,
 'gen_anychar_66': False,
 'gen_anychar_67': False,
 'gen_anychar_68': False,
 'gen_anychar_69': False,
 'gen_anychar_70': False,
 'gen_anychar_71': False,
 'gen_anychar_72': False,
 'gen_anychar_73': False,
 'gen_anychar_74': False,
 'gen_anychar_75': False,
 'gen_anychar_76': False,
 'gen_anychar_77': False,
 'gen_anychar_78': False,
 'gen_anychar_79': False,
 'gen_anychar_80': False,
 'gen_anychar_81': False,
 'gen_anychar_82': False,
 'gen_anychar_83': False,
 'gen_anychar_84': False,
 'gen_anychar_85': False,
 'gen_anychar_86': False,
 'gen_anychar_87': False,
 'gen_anychar_88': False,
 'gen_anychar_89': False,
 'gen_anychar_90': False,
 'gen_anychar_91': False,
 'gen_anychar_92': False,
 'gen_anychar_93': False,
 'gen_anychar_94': False,
 'gen_anychar_95': False,
 'gen_anychar_96': False,
 'gen_anychar_97': False,
 'gen_anychar_98': False,
 'gen_anychar_99': False,
 'gen_anychars_0': False,
 'gen_uqvalue_0': False,
 'gen_uqchar_0': False,
 'gen_uqchar_1': False,
 'gen_uqchar_2': False,
 'gen_uqchar_3': False,
 'gen_uqchar_4': False,
 'gen_uqchar_5': False,
 'gen_uqchar_6': False,
 'gen_uqchar_7': False,
 'gen_uqchar_8': False,
 'gen_uqchar_9': False,
 'gen_uqchar_10': False,
 'gen_uqchar_11': False,
 'gen_uqchar_12': False,
 'gen_uqchar_13': False,
 'gen_uqchar_14': False,
 'gen_uqchar_15': False,
 'gen_uqchar_16': False,
 'gen_uqchar_17': False,
 'gen_uqchar_18': False,
 'gen_uqchar_19': False,
 'gen_uqchar_20': False,
 'gen_uqchar_21': False,
 'gen_uqchar_22': False,
 'gen_uqchar_23': False,
 'gen_uqchar_24': False,
 'gen_uqchar_25': False,
 'gen_uqchar_26': False,
 'gen_uqchar_27': False,
 'gen_uqchar_28': False,
 'gen_uqchar_29': False,
 'gen_uqchar_30': False,
 'gen_uqchar_31': False,
 'gen_uqchar_32': False,
 'gen_uqchar_33': False,
 'gen_uqchar_34': False,
 'gen_uqchar_35': False,
 'gen_uqchar_36': False,
 'gen_uqchar_37': False,
 'gen_uqchar_38': False,
 'gen_uqchar_39': False,
 'gen_uqchar_40': False,
 'gen_uqchar_41': False,
 'gen_uqchar_42': False,
 'gen_uqchar_43': False,
 'gen_uqchar_44': False,
 'gen_uqchar_45': False,
 'gen_uqchar_46': False,
 'gen_uqchar_47': False,
 'gen_uqchar_48': False,
 'gen_uqchar_49': False,
 'gen_uqchar_50': False,
 'gen_uqchar_51': False,
 'gen_uqchar_52': False,
 'gen_uqchar_53': False,
 'gen_uqchar_54': False,
 'gen_uqchar_55': False,
 'gen_uqchar_56': False,
 'gen_uqchar_57': False,
 'gen_uqchar_58': False,
 'gen_uqchar_59': False,
 'gen_uqchar_60': False,
 'gen_uqchar_61': False,
 'gen_uqchar_62': False,
 'gen_uqchar_63': False,
 'gen_uqchar_64': False,
 'gen_uqchar_65': False,
 'gen_uqchar_66': False,
 'gen_uqchar_67': False,
 'gen_uqchar_68': False,
 'gen_uqchar_69': False,
 'gen_uqchar_70': False,
 'gen_uqchar_71': False,
 'gen_uqchar_72': False,
 'gen_uqchar_73': False,
 'gen_uqchar_74': False,
 'gen_uqchar_75': False,
 'gen_uqchar_76': False,
 'gen_uqchar_77': False,
 'gen_uqchar_78': False,
 'gen_uqchar_79': False,
 'gen_uqchar_80': False,
 'gen_uqchar_81': False,
 'gen_uqchar_82': False,
 'gen_uqchar_83': False,
 'gen_uqchar_84': False,
 'gen_uqchar_85': False,
 'gen_uqchar_86': False,
 'gen_uqchar_87': False,
 'gen_uqchar_88': False,
 'gen_uqchars_0': False,
 'gen_attributes_0': False,
 'gen_attributes_1': True,
 'gen_space_0': False,
 'gen_space_1': False,
 'gen_space_2': False,
 'gen_a_content_1_0': False,
 'gen_a_content_1_1': True,
 'gen_address_content_1_0': False,
 'gen_address_content_1_1': True,
 'gen_param_1_0': False,
 'gen_param_1_1': True,
 'gen_block_content_1_0': False,
 'gen_block_content_1_1': True,
 'gen_body_content_1_0': False,
 'gen_body_content_1_1': True,
 'gen_body_content_2_0': False,
 'gen_body_content_2_1': True,
 'gen_body_content_3_0': False,
 'gen_body_content_3_1': True,
 'gen__r__1_0': False,
 'gen__r__1_1': True,
 'gen_li_tag_1_0': True,
 'gen_li_tag_1_1': True,
 'gen_dl_content_1_0': True,
 'gen_dl_content_1_1': True,
 'gen_legend_tag_1_0': False,
 'gen_legend_tag_1_1': True,
 'gen_form_content_1_0': False,
 'gen_form_content_1_1': True,
 'gen_flow_content_1_0': False,
 'gen_flow_content_1_1': True,
 'gen_form_content_2_0': False,
 'gen_form_content_2_1': True,
 'gen_frameset_content_1_0': False,
 'gen_frameset_content_1_1': True,
 'gen_head_content_1_0': False,
 'gen_head_content_1_1': True,
 'gen_label_content_1_0': False,
 'gen_label_content_1_1': True,
 'gen_area_1_0': False,
 'gen_area_1_1': True,
 'gen_li_tag_2_0': False,
 'gen_li_tag_2_1': True,
 'gen_body_content_4_0': False,
 'gen_body_content_4_1': True,
 'gen_param_2_0': False,
 'gen_param_2_1': True,
 'gen_li_tag_3_0': True,
 'gen_li_tag_3_1': True,
 'gen_option_tag_1_0': False,
 'gen_option_tag_1_1': True,
 'gen_plain_text_1_0': False,
 'gen_plain_text_1_1': True,
 'gen_entity_1_0': False,
 'gen_entity_1_1': True,
 'gen_pre_content_1_0': False,
 'gen_pre_content_1_1': True,
 'gen_select_content_1_0': False,
 'gen_select_content_1_1': True,
 'gen_caption_tag_1_0': False,
 'gen_caption_tag_1_1': True,
 'gen_colgroup_tag_1_0': False,
 'gen_colgroup_tag_1_1': True,
 'gen_table_content_1_0': False,
 'gen_table_content_1_1': True,
 'gen_text_content_1_0': False,
 'gen_text_content_1_1': True,
 'gen_table_cell_1_0': False,
 'gen_table_cell_1_1': True,
 'gen_li_tag_4_0': False,
 'gen_li_tag_4_1': True,
 'gen_space_1_0': False,
 'gen_space_1_1': True,
 'gen_attributes_1_0': False,
 'gen_attributes_1_1': True,
 'gen_space_2_0': False,
 'gen_space_2_1': True,
 'gen_anychar_1_0': False,
 'gen_anychar_1_1': True,
 'gen_uqchar_1_0': False,
 'gen_uqchar_1_1': True,
 'gen_space_3_0': False,
 'gen_space_3_1': True}

In [ ]:
my_grammar = json_grammar

Maximum depth is 7; beyond that the size goes > 50G

## Existing tools
* Grammarinator (Python based)
* Gramfuzz (Python based)
* Dharma (Python based)

In [ ]:
class Sanitize:
    def __init__(self, g):
        self.g = g
  
    def to_key(self, k):
        s = k.replace('-', '_')
        s = s.replace('[', 'Osq').replace(']','Csq')
        s = s.replace('{','Obr').replace('}','Cbr')
        s = s.replace('import','XimportX')
        s = s.replace('class', 'XclassX')
        s = s.replace('def', 'XdefX')
        return s

    def to_token(self, t):
        return t
    
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        # these should not matter for performance comparisons,
        # and makes my life simpler
        esc = {'\r': '\r', '\n': '\n',
             '\\': '\\',
             '"':'"',
             "'":"'"}
        for i in t:
            if i in esc:
                my_tokens.append(esc[i])
            else:
                my_tokens.append(i)
        return my_tokens
            
        return list(t)

    def to_rule(self, rule, grammar):
        tokens = [k for t in rule for k in self.split_tokens(t, grammar)]
        return [self.to_token(t) if t not in grammar else self.to_key(t)
                for t in tokens]

    def translate(self):
        new_grammar = {}
        for k in self.g:
            rules = self.g[k]
            new_grammar[self.to_key(k)] = [self.to_rule(rule, self.g) for rule in rules]
        return new_grammar

### Grammarinator

Be sure to install the latest from git. The --random-seed argument is only in git HEAD.

In [ ]:
ipkg('grammarinator', 'git+https://github.com/renatahodovan/grammarinator.git')

In [ ]:
class AntlrG(Sanitize):
    def to_key(self, k):
        return super().to_key(k)[1:-1]

    def esc_token(self, t):
        # these are multi-char tokens
        t = t.replace('\\','\\\\')
        t = t.replace("'","\\\'")
        t = t.replace('\n','\\n')
        t = t.replace('\r','\\r')
        t = t.replace('\t','\\t')
        return t

    def rule_to_s(self, rule, grammar):
        return ' '.join(["'%s'" % self.esc_token(t)
                         if t not in grammar else self.to_key(t)
                         for t in rule])

    def translate(self):
        lines = ['grammar Grammar;']
        for k in self.g:
            rules = self.g[k]
            v = '\n    |'.join([self.rule_to_s(rule, self.g)
                                for rule in rules])
            lines.append('''\
%s
    : %s
    ;''' % (self.to_key(k), v))
        return '\n'.join(lines)

In [ ]:
g4 = AntlrG(my_grammar).translate()

In [ ]:
with open('testers/grammar.g4', 'w+') as f:
    print(g4, file=f)

In [ ]:
!grammarinator-process testers/grammar.g4 -o testers

In [ ]:
glexer = 'testers/GrammarUnlexer.py'
gparser = 'testers/GrammarUnparser.py'

In [ ]:
!rm -rf tests

In [ ]:
!grammarinator-generate -l testers/GrammarUnlexer.py -p testers/GrammarUnparser.py -r start -n 10 -o tests/ -j 1 --sys-recursion-limit 20900 -d 10 --random-seed 0

In [ ]:
import pathlib

In [ ]:
class GrammarinatorTester(Tester):
    def show_files(self, path):
        tests = pathlib.Path(path)
        for tf in tests.glob('*'):
            with open(tf) as f:
                print(repr(f.read()))

In [ ]:
GrammarinatorTester().show_files('tests')

In [ ]:
class GrammarinatorTester(GrammarinatorTester):
    def folder_size(self, path='.'):
        def cksum(fn):
            v = !cksum {fn}
            return (v.fields(0)[0])
            #with open(fn) as f: return f.read()
        total = 0
        num = 0
        ufiles = set()
        for entry in os.scandir(path):
            if entry.is_file():
                total += entry.stat().st_size
                num += 1
                #ufiles.add(cksum(entry.path))
            elif entry.is_dir():
                raise Exception('Only flat directories expected now.') 
                # total += self.folder_size(entry.path)
        return total, num

In [ ]:
GrammarinatorTester().folder_size('tests')

In [ ]:
class GrammarinatorTester(GrammarinatorTester):
    def init_run(self):
        super().init_run()
        !grammarinator-process testers/grammar.g4 -o testers/
        
    def pre_time(self):
        super().pre_time()
    
    def exec_program(self, seed, max_depth, t):
        glexer = 'testers/GrammarUnlexer.py'
        gparser = 'testers/GrammarUnparser.py'
        # seed, maxnum, max_depth
        return f"grammarinator-generate -l {glexer} -p {gparser}  --random-seed {seed} -n {self.max_num} -d {max_depth}  -r start  -o tests/ -j 1 --sys-recursion-limit 20900"

    def post_time(self):
        super().post_time()
        self.size, self.lines = self.folder_size(path='tests/')
        !rm -rf tests

In [ ]:
GrammarinatorTester().run_test().show()

### GramFuzz

The fork at `vrthra-forks` contains a few fixes for python 2to3 which have not made to the main repo yet.

In [ ]:
ipkg('gramfuzz', 'git+https://github.com/vrthra-forks/gramfuzz.git')

In [ ]:
class RDefG(AntlrG):
    def to_key(self, k):
        return 'RRef("%s")' % super().to_key(k)
 
    def esc_token(self, t):
        # these are multi-char tokens
        t = t.replace('\\','\\\\')
        t = t.replace('"','\\\"') # main difference from Antlr -- Quote char.
        t = t.replace('\n','\\n')
        t = t.replace('\r','\\r')
        t = t.replace('\t','\\t')
        return t

    def rule_to_s(self, rule, grammar):
        if len(rule) == 0: return '""'
        s =['"%s"' % self.esc_token(t)
                         if t not in grammar else 'RRef("%s")' % t
                         for t in rule]
        if len(s) == 1: return s[0]
        return 'And(%s)' % ', '.join(s)

    def translate(self):
        lines = ['''\
from gramfuzz.fields import*
TOP_CAT = "grammar"
class RDef(Def): cat="grammar-def"
class RRef(Ref): cat="grammar-def"

# top-level rule
Def("grammar", RRef("<start>", cat="grammar-def") )
''']
        for key in self.g:
            rules = self.g[key]
            if len(rules) == 1:
                srules = self.rule_to_s(rules[0], self.g)
            else:
                srules = "Or(%s)" % ' ,'.join(sorted([self.rule_to_s(rule, self.g) for rule in rules], reverse=True))
            lines.append('''\
RDef("%(key)s",
%(rules)s
)
''' % {'key':key, 'rules':srules})
        return '\n'.join(lines)

In [ ]:
roman_grammar = {
    '<start>' : [['<roman>']],
    '<roman>' : [['<hundreds>','<tens>','<units>']],
    '<hundreds>' : [['<lowhundreds>'],['CD'],['D','<lowhundreds>'],['CM']],
    '<lowhundreds>' : [[],['<lowhundreds>','C']],
    '<tens>' : [['<lowtens>'],['XL'], ['<lowtens>'], ['XC']],
    '<lowtens>' : [[],['<lowtens>','X']],
    '<units>' : [['<lowunits>'],['IV'], ['V', '<lowunits>'],['IX']],
    '<lowunits>' : [[],['<lowunits>', 'I']]
}

In [ ]:
with open('testers/roman_rdef.py', 'w+') as f:
    print(RDefG(roman_grammar).translate(), file=f)
!cat testers/roman_rdef.py

In [ ]:
import gramfuzz
fuzzer = gramfuzz.GramFuzzer()
fuzzer.load_grammar("testers/roman_rdef.py")
names = fuzzer.gen(cat="default", num=10)
print("\n".join(names))

In [ ]:
with open('testers/grammar_rdef.py', 'w+') as f:
    print(RDefG(my_grammar).translate(), file=f)
!cat testers/grammar_rdef.py

In [ ]:
with open('testers/gram_fuzz.py', 'w+') as f:
    print('''\
import gramfuzz, sys
gramfuzz.rand.seed(int(sys.argv[2]))
fuzzer = gramfuzz.GramFuzzer()
fuzzer.load_grammar(sys.argv[1])
names = fuzzer.gen(cat="default",
                   num=int(sys.argv[3]),
                   max_recursion=int(sys.argv[4]))
for n in names:
   print(repr(n))''', file=f)

In [ ]:
!python testers/gram_fuzz.py testers/grammar_rdef.py 0 10 10

In [ ]:
class GramFuzzTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"python testers/gram_fuzz.py testers/grammar_rdef.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
GramFuzzTester().run_test().show()

### Dharma

In [ ]:
ipkg('dharma', 'dharma')

In [ ]:
class DharmaG(AntlrG):
    def esc_token(self, t):
        if t.strip() == '':
            return '%range%( - )'
        # these are multi-char tokens
        #t = t.replace('\\','\\\\')
        #t = t.replace('"','\\\"')
        t = t.replace('\n','') # dont know what to do
        t = t.replace('\r','') # dont know what to do
        t = t.replace('\t','') # dont know what to do
        return t
    
    def rule_to_s(self, rule, grammar):
        if len(rule) == 0: return '""'
        s =['"%s"' % self.esc_token(t)
                         if t not in grammar else 'RRef("%s")' % t
                         for t in rule]
        if len(s) == 1: return s[0]
        return 'And(%s)' % ', '.join(s)

    def rule_to_s(self, rule, grammar):
        if len(rule) == 0:
            # *note* Dharma does not let us define epsilon rules. So we have to make do with
            # generating whitespace.
            return '%range%( - )'
        return ''.join([self.esc_token(t)
                        if t not in grammar else '+%s+' % self.to_key(t)
                         for t in rule])

    def translate(self):
        lines = ['''%%% Dharma Grammar
        ''']
        for k in self.g:
            rules = self.g[k]
            v = '\n    '.join([s for s in [self.rule_to_s(rule, self.g)
                                for rule in rules] if s.strip() != ''])
            lines.append('''\
%(key)s :=
    %(rules)s
'''% {'key':self.to_key(k), 'rules':v})
        lines.append(''' 
%section% := variance
main :=
    +start+
''')
        return '\n'.join(lines)

In [ ]:
with open('testers/grammar.dg', 'w+') as f:
    print(DharmaG(my_grammar).translate(), file=f)
!cat testers/grammar.dg

In [ ]:
!rm -rf tests
!python -m dharma -grammars testers/grammar.dg  -count 10 -seed 200 -storage tests/ -format txt

In [ ]:
!cat tests/*.txt

In [ ]:
class DharmaTester(GrammarinatorTester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"python -m dharma -grammars testers/grammar.dg -seed {seed} -count {self.max_num} -logging 30 -storage tests/"

In [ ]:
DharmaTester().run_test().show()

### The Fuzzing book approach

In [ ]:
def pp_grammar(grammar): return json.dumps(grammar, indent=2, sort_keys=False)

In [ ]:
class Trans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        esc = {'\r': '\\\r', '\n': '\\\n',
             '\\': '\\\\',
             '"':'\\\"',
             "'":"'"}
        for i in t:
            if i in esc:
                my_tokens.append(esc[i])
            else:
                my_tokens.append(i)
        return my_tokens
            
        return list(t)

    def to_rule(self, rule, grammar):
        tokens = [k for t in rule for k in self.split_tokens(t, grammar)]
        return [self.to_token(t) if t not in grammar else self.to_key(t)
                for t in tokens]

    def translate(self):
        new_grammar = {}
        for k in self.g:
            rules = self.g[k]
            new_grammar[self.to_key(k)] = [self.to_rule(rule, self.g) for rule in rules]
        return new_grammar

In [ ]:
s_grammar = Sanitize(my_grammar).translate()

In [ ]:
s_grammar

In [ ]:
with open('testers/fuzzingbook_gfuzzer.py', 'w+') as f:
    print('''grammar = ''', pp_grammar(s_grammar), file=f)
    print("""
result = ''
from fuzzingbook.GrammarFuzzer import GrammarFuzzer
def canonical(grammar):
    new_g = {}
    for k in grammar:
        new_g[k] = []
        for rule in grammar[k]:
            new_g[k].append(''.join(rule))
    return new_g
import random
def main(args):
    random.seed(int(sys.argv[1]))
    global result
    max_num = int(args[2])
    max_depth = int(args[3])
    fuzzer = GrammarFuzzer(canonical(grammar), max_nonterminals=max_depth)
    global result
    for i in range(max_num):
        result = fuzzer.fuzz()
        print(result)
        result = ''
import sys
main(sys.argv)""", file=f)

In [ ]:
!cat testers/fuzzingbook_gfuzzer.py

In [ ]:
!python testers/fuzzingbook_gfuzzer.py 0 10 10

In [ ]:
class FuzzingbookTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"python testers/fuzzingbook_gfuzzer.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
# The fuzzing book fuzzer does not improve with more depth.
FuzzingbookTester(limit_depth=5).run_test().show()

## Building a grammar fuzzer

In [ ]:
import sys
assert sys.platform == 'darwin'
sys.setrecursionlimit(20900) # for OSX only

In [ ]:
class Fuzzer:
    def __init__(self, grammar):
        self.grammar = grammar
    
    def fuzz(self, key='<start>', max_num=None, max_depth=None):
        raise NotImplemented()

In [ ]:
class NFuzzer(Fuzzer):
    def gen_key(self, key):
        return (self.gen_rule(random.choice(self.grammar[key]))
                if key in self.grammar else key)

    def gen_rule(self, rule):
        return ''.join(self.gen_key(token) for token in rule)

    def fuzz(self, key='<start>', max_depth=None):
        return self.gen_key(key)

In [ ]:
my_fuzzer = NFuzzer(s_grammar)
try:
    for i in range(100):
        print(repr(my_fuzzer.fuzz()))
except:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(exc_type, exc_value)

In [ ]:
import inspect
import json

In [ ]:
def get_opts(args, log=False):
    seed = int(args[1])
    max_num = int(args[2])
    max_depth = int(args[3])
    random.seed(seed)
    sys.setrecursionlimit(20900)
    if log:
        print("seed=%d, num=%d, depth=%d" % (seed, max_num, max_depth), file=sys.stderr)
    return max_num, max_depth

In [ ]:
def extract_class_definition(cls, log=False):
    eldest = [c for c in cls.mro()
                if c.__name__ == cls.__name__ and
                   cls.__name__ not in {i.__name__ for i in c.__bases__}]
    n_parents = sum([[j.__name__ for j in i.__bases__] for i in eldest], [])
    s_parents = '(%s)' % ', '.join(set(n_parents)) if n_parents else ''
    buf = ["class %s%s:" % (cls.__name__, s_parents)]
    seen = set()
    i = 0
    for curcls in cls.mro():
        i += 1
        if log: print('Parent: %d' % i, curcls.__name__)
        if curcls.__name__ != cls.__name__: continue
        for fn_name in dir(curcls):
            if log: print('\t:', fn_name)
            if fn_name in seen: continue
            if fn_name == '__new__':
                continue
            fn = curcls.__dict__.get(fn_name)
            if fn is None:
                continue
            if ('function' in str(type(fn))):
                seen.add(fn_name)
                buf.append(inspect.getsource(fn))
    return '\n'.join(buf)

In [ ]:
def write_file(file_name, grammar, classes, fns=[get_opts], fuzzer=None):
    with open(file_name, 'w+') as f:
        print('''grammar = ''', pp_grammar(grammar), file=f)
        for cls in classes:
            print(extract_class_definition(cls), file=f)
        for fn in fns:
            print(inspect.getsource(fn), file=f)
        print("""
import itertools
import sys
import random
def main(args):
    max_num, max_depth = get_opts(args)
    my_fuzzer = %s(grammar)
    for i in range(max_num):
        print(my_fuzzer.fuzz(key='<start>', max_depth=max_depth))
try:
    main(sys.argv)
    sys.exit(0)
except RecursionError as e:
    print(e, file=sys.stderr)
    sys.exit(2)
""" % fuzzer.__name__, file=f)

In [ ]:
write_file('testers/grammar_producer_naive.py', s_grammar, [Fuzzer, NFuzzer], fuzzer=NFuzzer)

In [ ]:
!cat testers/grammar_producer_naive.py

In [ ]:
# the seed and max_num is chosen to avoid recusion error.
!time python testers/grammar_producer_naive.py 1 96 0 > testers/fuzz.out

## Setting expansion limits

We can compute the least cost paths to take and use only those paths after a given depth is exceeded.

In [ ]:
import sys
import functools

In [ ]:
class LimitFuzzer(Fuzzer):
    def symbol_cost(self, grammar, symbol, seen):
        if symbol in self.key_cost: return self.key_cost[symbol]
        if symbol in seen:
            self.key_cost[symbol] = float('inf')
            return float('inf')
        v = min((self.expansion_cost(grammar, rule, seen | {symbol})
                    for rule in grammar.get(symbol, [])), default=0)
        self.key_cost[symbol] = v
        return v

    def expansion_cost(self, grammar, tokens, seen):
        return max((self.symbol_cost(grammar, token, seen)
                    for token in tokens if token in grammar), default=0) + 1

In [ ]:
class LimitFuzzer(LimitFuzzer):
    def gen_key(self, key, depth, max_depth):
        if key not in self.grammar: return key
        if depth > max_depth:
            clst = sorted([(self.cost[key][str(rule)], rule) for rule in self.grammar[key]])
            rules = [r for c,r in clst if c == clst[0][0]]
        else:
            rules = self.grammar[key]
        return self.gen_rule(random.choice(rules), depth+1, max_depth)

    def gen_rule(self, rule, depth, max_depth):
        return ''.join(self.gen_key(token, depth, max_depth) for token in rule)

    def fuzz(self, key='<start>', max_depth=10):
        return self.gen_key(key=key, depth=0, max_depth=max_depth)

In [ ]:
class LimitFuzzer(LimitFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        self.key_cost = {}
        self.cost = self.compute_cost(grammar)
 
    def compute_cost(self, grammar):
        cost = {}
        for k in grammar:
            cost[k] = {}
            for rule in grammar[k]:
                cost[k][str(rule)] = self.expansion_cost(grammar, rule, set())  
        return cost

In [ ]:
my_fuzzer = LimitFuzzer(s_grammar)

In [ ]:
my_fuzzer.fuzz()

In [ ]:
write_file('testers/grammar_producer_limit.py', s_grammar, [Fuzzer, LimitFuzzer], fuzzer=LimitFuzzer)

In [ ]:
!cat testers/grammar_producer_limit.py

In [ ]:
class PyLimitTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"python ./testers/grammar_producer_limit.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
!python testers/grammar_producer_limit.py 0 10 10

In [ ]:
PyLimitTester().run_test().show()

## Using precomputed string pools

**idea**: We can precompute the closing parts.

In [ ]:
class PooledFuzzer(LimitFuzzer):
    def compute_cost(self, grammar, cost={}):
        return {k:sorted([(self.expansion_cost(grammar, rule, set()), rule)
                          for rule in grammar[k]])
                for k in self.grammar}

In [ ]:
class PooledFuzzer(PooledFuzzer):
    def cheap_grammar(self):
        new_grammar = {}
        for k in self.cost:
            crules = self.cost[k]
            min_cost = crules[0][0]
            new_grammar[k] = [r for c,r in crules if c == min_cost]
            assert len(new_grammar[k]) > 0
        return new_grammar

In [ ]:
PooledFuzzer(s_grammar).cheap_grammar()

In [ ]:
import itertools
import random

In [ ]:
class PooledFuzzer(PooledFuzzer):
    def get_strings_for_key(self, grammar, key='<start>'):
        if key not in grammar: return [key]
        v = sum([self.get_strings_for_rule(grammar, rule)
                 for rule in grammar[key]], [])
        return random.sample(v, min(self.MAX_SAMPLE, len(v)))

    def get_strings_for_rule(self, grammar, rule):
        my_strings_list = [self.get_strings_for_key(grammar, key) for key in rule]
        v = [''.join(l) for l in itertools.product(*my_strings_list)]
        return random.sample(v, min(self.MAX_SAMPLE, len(v)))

    def completion_strings(self):
        # we are being choosy
        return {k:self.get_strings_for_key(self.c_grammar, k)
                for k in self.c_grammar}

In [ ]:
pf = PooledFuzzer(s_grammar)

In [ ]:
pf.c_grammar = pf.cheap_grammar()

In [ ]:
pf.MAX_SAMPLE = 255
strings = pf.completion_strings()

In [ ]:
for k in strings:
    print(k, strings[k])

In [ ]:
class PooledFuzzer(PooledFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        self.c_grammar = self.cheap_grammar()
        self.MAX_SAMPLE = 255
        self.pool_of_strings = self.completion_strings()
        # reorder our grammar rules by cost.
        for k in self.grammar:
            self.grammar[k] = [r for (i,r) in self.cost[k]]
        self.ordered_grammar = True
        
    def gen_key(self, key, depth, max_depth):
        if key not in self.grammar: return key
        if depth > max_depth:
            return random.choice(self.pool_of_strings[key])
        return self.gen_rule(random.choice(self.grammar[key]), depth+1, max_depth)

In [ ]:
my_fuzzer = PooledFuzzer(s_grammar)
for i in range(10):
    print(repr(my_fuzzer.fuzz()))

In [ ]:
write_file('testers/grammar_producer_pool.py', s_grammar, [Fuzzer, LimitFuzzer, PooledFuzzer], fuzzer=PooledFuzzer)

In [ ]:
!cat testers/grammar_producer_pool.py

In [ ]:
class PyPooledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"python testers/grammar_producer_pool.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
PyPooledTester().run_test().show()

Can we do better?

## Compile the Grammar

In [ ]:
class PyTrans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        for i in t:
            my_tokens.append(i)
        return my_tokens

In [ ]:
pyc_grammar = PyTrans(my_grammar).translate()

In [ ]:
pyc_grammar

### Compile to Python

In [ ]:
# not clear what is the fastest: + or ''.join
# https://stackoverflow.com/questions/1316887/what-is-the-most-efficient-string-concatenation-method-in-python
class PyCompiledFuzzer(PooledFuzzer):
    def add_indent(self, string, indent):
        return '\n'.join([indent + i for i in string.split('\n')])

    # used for escaping inside strings
    def esc(self, t):
        t = t.replace('\\', '\\\\')
        t = t.replace('\n', '\\n')
        t = t.replace('\r', '\\r')
        t = t.replace('\t', '\\t')
        t = t.replace('\b', '\\b')
        t = t.replace('\v', '\\v')
        t = t.replace('"', '\\"')
        return t
    
    def esc_char(self, t):
        assert len(t) == 1
        t = t.replace('\\', '\\\\')
        t = t.replace('\n', '\\n')
        t = t.replace('\r', '\\r')
        t = t.replace('\t', '\\t')
        t = t.replace('\b', '\\b')
        t = t.replace('\v', '\\v')
        t = t.replace("'", "\\'")
        return t

    def k_to_s(self, k): return k[1:-1].replace('-', '_')

    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                res.append('''\
gen_%s(next_depth, max_depth)''' % self.k_to_s(token))
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return '\n'.join(res)

    def string_pool_defs(self):
        result =[]
        for k in self.pool_of_strings:
            result.append('''\
pool_of_%(key)s = %(values)s''' % {
                'key':self.k_to_s(k),
                'values': self.pool_of_strings[k]})
        result.append('''
result = []''')
        return '\n'.join(result)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
def main(args):
    global result
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start(0, max_depth)
        print(''.join(result))
        result = []
 
main(sys.argv)''')
        return '\n'.join(result)

    def gen_alt_src(self, key):
        rules = self.grammar[key]
        result = []
        result.append('''
def gen_%(name)s(depth, max_depth):
    next_depth = depth + 1
    if depth > max_depth:
        result.append(random.choice(pool_of_%(name)s))
        return
    val = random.randrange(%(nrules)s)''' % {
            'name':self.k_to_s(key),
            'nrules':len(rules)})
        for i, rule in enumerate(rules):
            result.append('''\
    if val == %d:
%s
        return''' % (i, self.add_indent(self.gen_rule_src(rule, key, i),'        ')))
        return '\n'.join(result)

    def gen_fuzz_src(self):
        result = []
        result.append(self.string_pool_defs())
        for key in self.grammar:
            result.append(self.gen_alt_src(key))
        return '\n'.join(result)

    def fuzz_src(self, key='<start>'):
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [ ]:
with open('testers/grammar_producer_pycompiled.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PyCompiledFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [ ]:
!cat testers/grammar_producer_pycompiled.py

In [ ]:
!python testers/grammar_producer_pycompiled.py 0 10 10

In [ ]:
class PyCompiledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"python testers/grammar_producer_pycompiled.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
PyCompiledTester().run_test().show()

In [ ]:
class PyRecCompiledFuzzer(PyCompiledFuzzer):
    def __init__(self, grammar):
        super().__init__(grammar)
        assert self.ordered_grammar
        self.rec_cost = {}
        self.compute_rule_recursion()

    def kr_to_s(self, key, i): return 'gen_%s_%d' % (self.k_to_s(key), i)
    # the grammar needs to be ordered by the cost.
    # else the ordering will change at the end.
    
    def is_rule_recursive(self, rname, rule, seen):
        if not rule: return False
        if rname in seen:
            return False # reached another recursive rule without seeing this one
        for token in rule:
            if token not in self.grammar: continue
            for i,trule in enumerate(self.grammar[token]):
                rn = self.kr_to_s(token, i)
                if rn  == rname: return True
                if rn in seen: return False
                v = self.is_rule_recursive(rname, trule, seen | {rn})
                if v: return True
        return False
    
    def is_key_recursive(self, check, key, seen):
        if not key in self.grammar: return False
        if key in seen: return False
        for rule in self.grammar[key]:
            for token in rule:
                if token not in self.grammar: continue
                if token == check: return True
                v = self.is_key_recursive(check, token, seen | {token})
                if v: return True
        return False
    
    def compute_rule_recursion(self):
        if IS_HTML:   # TODO -- to much time -- only for HTML
            self.rule_recursion = HTML_RULE_RECURSION
            self.key_recursion = HTML_KEY_RECURSION
            return
        self.rule_recursion = {}
        for k in self.grammar:
            for i_rule,rule in enumerate(self.grammar[k]):
                n = self.kr_to_s(k, i_rule)
                self.rule_recursion[n] = self.is_rule_recursive(n, rule, set())
        self.key_recursion = {}
        for k in self.grammar:
            self.key_recursion[k] = self.is_key_recursive(k, k, set())

In [ ]:
PyRecCompiledFuzzer(pyc_grammar).rule_recursion

In [ ]:
PyRecCompiledFuzzer(pyc_grammar).key_recursion

### Partial Evaluation in Python

In [ ]:
class PEFuzzer(PyRecCompiledFuzzer):
    def pe_rule(self, rule, depth):
        res = []
        res.append('''\
# rule=%(rule)s len[%(len)d]''' % {'rule':str(rule), 'len':len(rule)})
        if not rule:
            res.append('''\
pass''')
        for token in rule:
            res.append('''\
# token=%(token)s''' % {'token':token})
            if token in self.grammar:
                if self.key_recursion[token]:
                    res.append('''\
# sc
gen_%(key)s(depth+%(depth)s, max_depth)''' % {'key':self.k_to_s(token),'depth':depth+2})
                else:
                    res.append(self.pe_key(token, depth=depth+1))
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return '\n'.join(res)
 
    def pe_key(self, key, depth):
        if depth == self.MAX_PE_DEPTH:
            return 'gen_%(key)s(depth+%(depth)d, max_depth)' % {'key':self.k_to_s(key), 'depth':depth+1}
        rules = self.grammar[key]
        result = ['''\
#* %(key)s begins
if depth + %(depth)d > max_depth:
    result.append(random.choice(pool_of_strings['%(key)s']))
else:''' % {'name':self.k_to_s(key), 'key':key,  'depth':depth+1}]
        if len(rules) == 0:
            result.append('''\
    # indent dummy''')
        elif len(rules) == 1:
            result.append('''\
    # indent inline''')
            result.append(self.add_indent(self.pe_rule(rules[0], depth),'    '))
        else:
            result.append('''\
    val = random.randrange(%(nrules)s)''' % {'nrules':len(rules)})
            result.append('''\
    # indent here<
    if False:
        pass # dummy''')
            assert len(rules) > 1
            for i, rule in enumerate(rules):
                result.append('''\
    elif val == %d:''' % i)               
                result.append(self.add_indent(self.pe_rule(rule, depth),'        '))
            result.append('''\
    # indent here>''')
        result.append('''\
#* %(key)s ends''' % {'key': key})
        return '\n'.join(result)

    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                if token == key:
                    res.append('''\
# not unrolling
gen_%s(depth+1, max_depth)''' % self.k_to_s(token))
                else:
                    res.append('''\
#indent -<''')
                    res.append(self.pe_key(token, depth=0))
                    res.append('''\
#indent >-''')
                    
            else:
                res.append('''\
result.append("%s")''' % self.esc(token))
        return self.add_indent('\n'.join(res), '            ')

    def string_pool_defs(self):
        result =[]
        result.append('''\
pool_of_strings = %s''' % pp_grammar(self.pool_of_strings))
        result.append('''
result = [];''')
        return '\n'.join(result)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
def main(args):
    global result
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start(0, max_depth)
        print(''.join(result))
        result = []
main(sys.argv)
    ''')
        return '\n'.join(result)

    def gen_alt_src(self, key):
        rules = self.grammar[key]
        result = []
        result.append('''
def gen_%(name)s(depth, max_depth):
    # %(name)s begins
    if depth > max_depth:
        result.append(random.choice(pool_of_strings['%(key)s']))
    else:
        val = random.randrange(%(nrules)s)''' % {'name':self.k_to_s(key), 'key':key, 'nrules':len(rules)})
        for i, rule in enumerate(rules):
            result.append('''\
        if val == %d:
%s
            return''' % (i, self.gen_rule_src(rule, key, i)))
        result.append('''
    # %(name)s ends
        ''')
        return '\n'.join(result)
    
    def fuzz_src(self, key='<start>'):
        self.MAX_PE_DEPTH = 4
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [ ]:
with open('testers/grammar_producer_pe.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PEFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [ ]:
!wc -l testers/grammar_producer_pe.py

In [ ]:
!python testers/grammar_producer_pe.py 0 10 10

In [ ]:
class PyPETester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"python testers/grammar_producer_pe.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
PyPETester().run_test().show()

### Supercompile in Python

In [ ]:
class PySuperCompiledFuzzer(PyRecCompiledFuzzer):    
    def supercompile_rule(self, key, rule, i_rule, depth):
        gen_name = self.kr_to_s(key, i_rule)
        if self.rule_recursion[gen_name]:
            self.current_lst.append(gen_name)
            return '''\
%(gen_name)s(depth_%(depth)d) # recursing''' % {'gen_name':gen_name, 'depth':depth}
        res = []
        if len(rule) == 0:
            res.append('pass')
        else:
            for token in rule:
                if token not in self.grammar:
                    res.append('''\
result.append("%s")''' % self.esc(token))
                else:
                    res.append(# no indent
                        self.supercompile_key(token,
                                              depth=(depth+1)))
        return '\n'.join(res)
    def supercompile_key_internal(self, key, trule, i_trule, depth):
        if depth > self.MAX_SUPERCOMPILE_DEPTH:
            self.current_lst.append(self.kr_to_s(key, i_trule))
            return '%(gen_name)s(depth_%(depth)d) #slimit*' % {
                        'gen_name':self.kr_to_s(key, i_trule), 'depth':depth}
        else:
            return self.supercompile_rule(key, trule, i_trule, depth=depth)
 
    def supercompile_key(self, key, depth):
        # Should check for MAX_SUPERCOMPILE_DEPTH
        # should first get the random number curresponding to
        # len(grammar[key]) then it should unroll that elif cond.
        if len(self.grammar[key]) == 0: return '' # no more jumping on the bed
        res = ['''\
if depth_%(depth)d > max_depth:
    result.append(random.choice(pool_of_%(key)s))
else:
    depth_%(d_1)d = depth + %(d_1)d'''%{
            'key':self.k_to_s(key), 'depth': depth, 'd_1': depth+1}]
        if len(self.grammar[key]) == 1:
            # we do not have to get the random number, and check for
            # equality first.
            i_trule, trule  = 0, self.grammar[key][0]
            res.append(self.add_indent(
                self.supercompile_key_internal(key, trule, i_trule, depth),
                '''\
    '''))
        else:
            # First get the random number, then compare and
            # unroll
            res.append('''\
    val = random.randrange(%(len_rules)d)
    if False: # dummy for elsif
        pass''' % {'len_rules': len(self.grammar[key])})
            for i_trule, trule in enumerate(self.grammar[key]):
                res.append('''\
    elif val == %(i_trule)d:''' % {'i_trule': i_trule})
                res.append(self.add_indent(
                    self.supercompile_key_internal(key, trule, i_trule, depth),
                    '''\
        '''))

        return '\n'.join(res)
   
    def gen_rule_src(self, rule, key, i_rule):
        res = ['''\
def %(gen_name)s(depth):
    if depth > max_depth:
        result.append(random.choice(pool_of_%(key)s))
    else:
        depth_%(d_1)d = depth + %(d_1)d''' % {
            'gen_name':self.kr_to_s(key,i_rule),
            'key':self.k_to_s(key),
            'depth':0, 'd_1': 1}]
        
        # These should be a sequence of getting randon numbers
        # and unrolling appropriately.
        for token in rule:
            if token not in self.grammar:
                res.append('''\
        result.append("%s")''' % self.esc(token))
            else:
                res.append(self.add_indent(
                    self.supercompile_key(token, depth=1), '''\
        '''))
        return '\n'.join(res)

    def gen_main_src(self):
        result = []
        result.append('''
import random
import sys
max_depth = 0
def main(args):
    global result, max_depth
    max_num, max_depth = get_opts(args)
    for i in range(max_num):
        gen_start_0(0)
        print(''.join(result))
        result = []
main(sys.argv)
    ''')
        return '\n'.join(result)
    
    def gen_fuzz_src(self):
        keys_used = {}
        result = [self.string_pool_defs()]
        key_defs = {}
        for key in self.grammar:
            for i,rule in enumerate(self.grammar[key]):
                self.current_lst = []
                ks = self.kr_to_s(key, i)
                keys_used[ks] = self.current_lst
                key_defs[ks] = self.gen_rule_src(rule, key, i)
        key_set = set(keys_used['gen_start_0']) | {'gen_start_0'}
        old_len = 0
        while old_len != len(key_set):
            old_len = len(key_set)
            key_set.update(k1 for k in list(key_set) for k1 in keys_used[k])
            
        for k in key_set:
            result.append(key_defs[k])
        return '\n'.join(result)

    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 100
        result = [self.gen_fuzz_src(),
                  self.gen_main_src()]
        return ''.join(result)

In [ ]:
with open('testers/grammar_producer_pysupercompiled.py', 'w+') as f:
    for fn in [get_opts]:
        print(inspect.getsource(fn), file=f)
    result = PySuperCompiledFuzzer(pyc_grammar).fuzz_src()
    print(result, file=f)

In [ ]:
!wc -l testers/grammar_producer_pysupercompiled.py

In [ ]:
!python testers/grammar_producer_pysupercompiled.py 0 10 10

In [ ]:
class PySupercompiledTester(Tester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"python testers/grammar_producer_pysupercompiled.py {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
PySupercompiledTester().run_test().show()

### Compile to C

In [ ]:
class CTrans(Sanitize):
    def split_tokens(self, t, grammar):
        if t in grammar: return [t]
        my_tokens = []
        esc = {
           '\r': '\\r',
           '\n': '\\n',
           '\t': '\\t',
           '\\': '\\\\',
        }
        for i in t:
            #if i in esc:
            #    my_tokens.append(esc[i])
            #else:
                my_tokens.append(i)
        return my_tokens

In [ ]:
c_grammar = CTrans(my_grammar).translate()

In [ ]:
c_grammar

We compile a grammar into a C program that produces from the grammar.  Just how fast can we be?

In [ ]:
class CFuzzer(PyRecCompiledFuzzer):    
    def cheap_chars(self, string):
        # to be embedded within single quotes
        escaped = {'t':'\t', 'n': '\n', "'": "\\'", "\\":"\\\\", 'r': '\r'}
        slst = []
        while string:
            c, *string = string
            if c in {'\\'}:
                c1, *string = string
                slst.append(escaped[c1])
            elif c in {"'"}:
                slst.append("\'")
            else:
                slst.append(c)
        return slst
    
    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                res.append('gen_%s(depth +1);' % self.k_to_s(token))
            else:
                res.append("out('%s');" % self.esc_char(token))
        return '\n        '.join(res)

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        cheap_strings = self.pool_of_strings[k]
        result = ['''
void gen_%(name)s(int depth) {
    if (depth > max_depth) {
        int val = map(%(num_cheap_strings)d);
        const char* str = pool_%(name)s[val];
        const int str_l = pool_l_%(name)s[val];
        for (int i = 0; i < str_l; i++) {
            out(str[i]);
        }
        return;
    }

    int val = map(%(nrules)d);
    switch(val) {''' % {'name':self.k_to_s(k), 'nrules':len(rules),
                        'num_cheap_strings': len(cheap_strings),
                       }]
        for i, rule in enumerate(rules):
            result.append('''
    case %d:
        %s
        break;''' % (i, self.gen_rule_src(rule, k, i)))
        result.append('''
    }
}
    ''')
        return '\n'.join(result)
    
    def string_pool_defs(self):
        result = []
        for k in self.grammar:
            cheap_strings = self.pool_of_strings[k]
            result.append('''
const char* pool_%(k)s[] =  {%(cheap_strings)s};
const int pool_l_%(k)s[] =  {%(cheap_strings_len)s};
        ''' % {'k':self.k_to_s(k),
               'cheap_strings': ', '.join(['"%s"' % self.esc(s) for s in cheap_strings]),
               'cheap_strings_len': ', '.join([str(len(s)) for s in cheap_strings])})
        return '\n'.join(result)

    
    def fn_fuzz_decs(self):
        result = []
        for k in self.grammar:
            result.append('''void gen_%s(int depth);''' % self.k_to_s(k))
        return '\n'.join(result)
    
    def fn_map_def(self):
        return '''
int map(int v) {
    return random() % v;
}
 '''    
    def fn_out_def(self):
        return '''
void out(const char s) {
    fputc(s, stdout);
}       
 '''

    def fuzz_hdefs(self):
        return '''
#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>
'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(const char s);'''
    
    def fuzz_rand_var_defs(self):
        return '''
int map(int v);'''
    def fuzz_stack_var_defs(self):
        return '''
extern int max_depth;'''

    def fuzz_var_defs(self):
        return '\n'.join([self.fuzz_out_var_defs(), self.fuzz_rand_var_defs(), self.fuzz_stack_var_defs()])

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''
    
    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__();
    }'''

    def fn_main_def(self):
        result = '''
int main(int argc, char** argv) {
    int seed, max_num;
%(input_frag)s
    //srandom(time(0));
    srandom(seed);
%(loop_frag)s
    return 0;
}''' % {'input_frag':self.fn_main_input_frag(),
        'loop_frag': self.fn_main_loop_frag()}
        return result
    
    def main_stack_var_defs(self):
        return '''
int max_depth = 0;'''
    
    def main_init_var_defs(self):
        return '''
void gen_init__();'''
    
    def main_var_defs(self):
        return '\n'.join([self.main_stack_var_defs(), self.main_init_var_defs()])
    
    def fuzz_fn_defs(self):
        result = []
        for key in self.grammar:
            result.append(self.gen_alt_src(key))
        return '\n'.join(result)
    
    def fuzz_entry(self):
        return '''
void gen_init__() {
    gen_start(0);
    out('\\n');
    return;
}'''

    def main_hdefs(self):
        return '''
#define _LARGEFILE64_SOURCE
#define _FILE_OFFSET_BITS 64

#include <stdlib.h>
#include <stdio.h>
#include <time.h>
#include <string.h>
'''

    def gen_main_src(self):
        return '\n'.join([self.main_hdefs(),
                          self.main_var_defs(),
                          self.fn_map_def(),
                          self.fn_out_def(),
                          self.fn_main_def()])
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          self.fuzz_fn_defs(),
                          self.fuzz_entry()])

    def fuzz_src(self, key='<start>'):
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
main_src, fuzz_src = CFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_c_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_c_main.c', 'w+') as f:
    print(main_src, file=f)

In [ ]:
!nl -ba testers/grammar_producer_c_fuzz.c

In [ ]:
!cat testers/grammar_producer_c_main.c

In [ ]:
%cd testers
!cc -Ofast grammar_producer_c_main.c grammar_producer_c_fuzz.c  -o grammar_producer_c
%cd ..

In [ ]:
!./testers/grammar_producer_c 0 10 10

In [ ]:
# II
class CTester(Tester):
    def __init__(self, name=None, max_num=10000, start_depth=3, limit_depth=9, timeout=3600, iterations=100):
        super().__init__(name, max_num, start_depth, limit_depth, timeout)

    def exec_program(self, seed, max_depth, t):
        # seed, maxnum, max_depth
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_c {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
CTester().run_test().show()

### Partial Evaluation in C

In [ ]:
class PECFuzzer(CFuzzer):
    def pe_rule(self, rule, seen, depth):
        res = []
        res.append('''\
/* rule=%(rule)s len[%(len)d]*/''' % {'rule': str(rule), 'len': len(rule)})
        if not rule:
            res.append('''
/*break;*/
            ''')
        for token in rule:
            res.append('''
/* token=%(token)s*/''' % {'token': token})
            if token in self.grammar:
                if token in seen:
                    res.append('''\
/* sc */
gen_%(key)s(depth+%(depth)d);''' % {'key': self.k_to_s(token), 'depth': depth+2})
                else:
                    res.append(self.pe_key(token, seen=(seen | {token}), depth=depth+1))
            else:
                res.append('''\
out('%s');''' % self.esc_char(token))
        return '\n'.join(res)
        
        
    def pe_key(self, key, seen, depth):
        if depth == self.MAX_PE_DEPTH:
            return 'gen_%(key)s(depth+%(depth)d);' % {'key': self.k_to_s(key), 'depth': depth+1}
        rules = self.grammar[key] # ordered by the cost
        cheap_strings = self.pool_of_strings[key] 
        # we haven't restricted map to 256 yet.
        result = ['''\
/* %(key)s begins*/
if ((depth + %(depth)d) > max_depth) {
    int val = map(%(num_cheap_strings)d);
    const char* str = pool_%(name)s[val];
    const int str_l = pool_l_%(name)s[val];
    for (int i = 0; i < str_l; i++) {
        out(str[i]);
    }
} else {''' %  {'name':self.k_to_s(key),
                'key': key,
                'depth': depth+1,
                'num_cheap_strings': len(cheap_strings)}]
        if len(rules) == 0:
            result.append('''\
    /*indent dummy*/
            ''')
        elif len(rules) == 1:
            result.append('''\
    /*indent inline*/
            ''')
            result.append(self.add_indent(self.pe_rule(rules[0], seen, depth),'    '))
        else:
            result.append('''\
    int val = map(%(nrules)d);
            ''' % {'nrules': len(rules)})
            result.append('''\
    switch(val) {
            ''')
            for i, rule in enumerate(rules):
                result.append('''\
    case %d:
        {''' % i)
                result.append(self.add_indent(self.pe_rule(rule, seen, depth),'            '))
                result.append('''\
            break;
        }''')
            result.append('''\
    }''')
        result.append('''\
}
/* %(key)s ends*/''' % {'key': key})
        return '\n'.join(result)
        
    def gen_rule_src(self, rule, key, i):
        res = []
        for token in rule:
            if token in self.grammar:
                if token == key:
                    res.append('''\
/* not unrolling*/
gen_%(key)s(depth +1);''' % {'key':self.k_to_s(token)})
                else:
                    res.append('''\
/*indent -<*/''')
                    res.append(self.pe_key(token, seen={key, token}, depth=0))
                    res.append('''\
/*indent >-*/''')
            else:
                res.append("out('%s');" % self.esc_char(token))
        return '\n'.join(res)
    
    def fuzz_src(self, key='<start>'):
        self.MAX_PE_DEPTH = 4
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
main_src, fuzz_src = PECFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_pec_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_pec_main.c', 'w+') as f:
    print(main_src, file=f)

In [ ]:
!wc -l testers/grammar_producer_pec_fuzz.c

In [ ]:
!cat testers/grammar_producer_pec_main.c

In [ ]:
%cd testers
!cc -Ofast grammar_producer_pec_main.c grammar_producer_pec_fuzz.c  -o grammar_producer_pec
%cd ..

In [ ]:
!./testers/grammar_producer_pec 0 10 10

In [ ]:
class CPETester(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_pec {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
CPETester().run_test().show()

### Supercompile C

In [ ]:
# II
class CSuperCompiledFuzzer(CFuzzer):

    def kr_to_s(self, key, i): return 'gen_%s_%d' % (self.k_to_s(key), i)
    
    def supercompile_rule(self, key, rule, i_rule, depth):
        gen_name = self.kr_to_s(key, i_rule)
        if self.rule_recursion[gen_name]:
            self.current_lst.append(gen_name)
            return '''\
%(gen_name)s(depth_%(depth)d); /* recurse*/''' % {'gen_name':gen_name, 'depth':depth}
        res = []
        if len(rule) == 0:
            res.append('/*pass*/')
        else:
            for token in rule:
                if token not in self.grammar:
                    res.append('''\
out('%s');''' % self.esc_char(token))
                else:
                    res.append(# no indent
                        self.supercompile_key(token,
                                              depth=(depth+1)))
        return '\n'.join(res)
    
    def supercompile_key_internal(self, key, trule, i_trule, depth):
        if depth > self.MAX_SUPERCOMPILE_DEPTH:
            self.current_lst.append(self.kr_to_s(key, i_trule))
            return '%(gen_name)s(depth_%(depth)d); /*slimit*/' % {
                        'gen_name':self.kr_to_s(key, i_trule), 'depth':depth}
        else:
            return self.supercompile_rule(key, trule, i_trule, depth=depth)
        
    def choose_from_cheap_strings(self, key):
        cheap_strings = self.pool_of_strings[key]
        if len(cheap_strings) == 1:
            return '\n'.join(["out('%s');" % c for c in self.cheap_chars(cheap_strings[0])])
        l = [len(s) for s in cheap_strings]
        if len(set(l)) == 1:
            name = ['''\
int val = map(%(num_cheap_strings)d);
const char* str = pool_%(name)s[val];''' % {
                'name':self.k_to_s(key), 'num_cheap_strings': len(cheap_strings)}]
            out = ["out(str[%d]);" % i for i in range(l[0])]
            return '\n'.join(name + out)
        else:
            return '''\
int val = map(%(num_cheap_strings)d);
const char* str = pool_%(name)s[val];
const int str_l = pool_l_%(name)s[val];
for (int i = 0; i < str_l; i++) {
    out(str[i]);
}
    '''%{
            'name':self.k_to_s(key),
            'num_cheap_strings': len(cheap_strings)}

    def supercompile_key(self, key, depth):
        # Should check for MAX_SUPERCOMPILE_DEPTH
        # should first get the random number curresponding to
        # len(grammar[key]) then it should unroll that elif cond.
        if len(self.grammar[key]) == 0: return '' # no more jumping on the bed

        res = ['''\
if (depth_%(depth)d > max_depth) {
%(select_from_pool)s
} else {
    int depth_%(d_1)d = depth + %(d_1)d;'''%{
            'select_from_pool': self.add_indent(self.choose_from_cheap_strings(key), '    '),
            'depth': depth,
            'd_1': depth+1}]
        if len(self.grammar[key]) == 1:
            # we do not have to get the random number, and check for
            # equality first.
            i_trule, trule  = 0, self.grammar[key][0]
            res.append(self.add_indent(
                self.supercompile_key_internal(key, trule, i_trule, depth),
                '''\
    '''))
        else:
            # First get the random number, then compare and
            # unroll
            res.append('''\
    int val = map(%(len_rules)d);
    switch(val) {''' % {'len_rules': len(self.grammar[key])})
            for i_trule, trule in enumerate(self.grammar[key]):
                res.append('''\
    case %(i_trule)d:
        {''' % {'i_trule': i_trule})
                res.append(self.add_indent(
                    self.supercompile_key_internal(key, trule, i_trule, depth),
                    '''\
            '''))
                res.append('''\
            break;
        } /*case %d*/''' % i_trule)
            res.append('''\
    }/*switch*/''')
        res.append('''\
}/*ifelse %d*/''' % depth)
        return '\n'.join(res)
   
    def gen_rule_src(self, rule, key, i_rule):
        res = ['''\
void %(gen_name)s(int depth) {
    if (depth > max_depth) {
%(select_from_pool)s
    } else {
        int depth_%(d_1)d = depth + %(d_1)d; ''' % {
            'gen_name':self.kr_to_s(key,i_rule),
            'select_from_pool': self.add_indent(self.choose_from_cheap_strings(key),'        '),
            'depth':0,
            'd_1': 1}]
        # These should be a sequence of getting randon numbers
        # and unrolling appropriately.
        for token in rule:
            if token not in self.grammar:
                res.append('''\
        out('%s');''' % self.esc_char(token))
            else:
                res.append(self.add_indent(
                    self.supercompile_key(token, depth=1), '''\
        '''))
        res.append('''\
    } /*else*/
} /* %s */''' % self.kr_to_s(key, i_rule))
        return '\n'.join(res)
    
    # ----  
 
    def fn_fuzz_decs(self):
        result = []
        for k in self.grammar:
            for i,r in enumerate(self.grammar[k]):
                result.append('void %(name)s(int depth);' % {'name':self.kr_to_s(k, i)})
        return '\n'.join(result)
    
    def fuzz_fn_defs(self):
        keys_used = {}
        result = []
        key_defs = {}
        for key in self.grammar:
            for i_rule,rule in enumerate(self.grammar[key]):
                self.current_lst = []
                ks = self.kr_to_s(key, i_rule)
                keys_used[ks] = self.current_lst
                key_defs[ks] = self.gen_rule_src(rule, key, i_rule)
        key_set = set(keys_used['gen_start_0']) | {'gen_start_0'}
        old_len = 0
        while old_len != len(key_set):
            old_len = len(key_set)
            key_set.update(k1 for k in list(key_set) for k1 in keys_used[k])
            
        for k in key_set:
            result.append(key_defs[k])
        return '\n'.join(result)
 
    def fuzz_entry(self):
        return '''
void gen_init__() {
    gen_start_0(0);
    out('\\n');
    return;
}'''
 
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 0
        return self.gen_main_src(), self.gen_fuzz_src()

Going full supercompilation (below) actually reduces the speed by a small %. It is not clear why.

In [ ]:
class CSuperCompiledFuzzer(CSuperCompiledFuzzer):
    def string_pool_defs(self): return ''
    
    def choose_from_cheap_strings(self, key):
        short = False
        cheap_strings = self.pool_of_strings[key]
        if len(cheap_strings) == 1:
            return '\n'.join(["out('%s');" % self.esc_char(c) for c in cheap_strings[0]])
        elif len(cheap_strings) == 0:
            return ''
        else:
            lst = ['''
int val = map(%(num_cheap_strings)d);
switch(val){'''% {
            'name':self.k_to_s(key),
            'num_cheap_strings': len(cheap_strings)}]
            for i in range(len(cheap_strings)):
                lst.append('''
case %d:
    {''' % i)
                lst.extend(["    out('%s');" % self.esc_char(c) for c in  cheap_strings[i]])
                lst.append('''
    break;
    }''')
            lst.append('''
}''')
            return '\n'.join(lst)

In [ ]:
main_src, fuzz_src = CSuperCompiledFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_superc_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/grammar_producer_superc_main.c', 'w+') as f:
    print(main_src, file=f)

In [ ]:
!wc -l testers/grammar_producer_superc_fuzz.c

In [ ]:
!cat testers/grammar_producer_superc_main.c

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
!./testers/grammar_producer_superc 0 10 10

In [ ]:
class CSupercompiledTester(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_superc {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
CSupercompiledTester().run_test().show() # 0

In [ ]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 1
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
CSupercompiledTester().run_test().show() # 1

In [ ]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 2
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
CSupercompiledTester().run_test().show() # 2

In [ ]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 3
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
CSupercompiledTester().run_test().show() # 3

In [ ]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 5
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
CSupercompiledTester().run_test().show() # 5

In [ ]:
class CSuperCompiledFuzzer(CFuzzer):
    def fuzz_src(self, key='<start>'):
        self.MAX_SUPERCOMPILE_DEPTH = 10
        return self.gen_main_src(), self.gen_fuzz_src()

In [ ]:
%cd testers
!cc -Ofast grammar_producer_superc_main.c grammar_producer_superc_fuzz.c  -o grammar_producer_superc
%cd ..

In [ ]:
CSupercompiledTester().run_test().show() # 10

## Making the random -> choices map faster.

### [Replace the division by mulitplication](//https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/)

For convenience, here is our map function.
```c
int map(int v) {
    return random() % v;
}
```

```c
uint32_t
__attribute__((always_inline))
map(uint32_t to) {
    uint32_t from = random();
    return ((uint64_t) from * (uint64_t) to) >> 32;
}
```

### Replace random() by a faster pseudo random.

[Xoshiro**](http://xoshiro.di.unimi.it/xoshiro128starstar.c)

In [ ]:
# https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/ 
class CFuzzerPRNG(CFuzzer):
    def fn_map_def(self):
        return '''
uint64_t next(void);
uint32_t map(uint32_t to) {
    uint32_t from = next();
    return ((uint64_t) from * (uint64_t) to) >> 32;
}

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}
'''

In [ ]:
main_src, fuzz_src = CFuzzerPRNG(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprng_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprng_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_cprng_main.c

In [ ]:
%cd testers
!cc -Ofast grammar_producer_cprng_main.c grammar_producer_cprng_fuzz.c -o grammar_producer_cprng
%cd ..

In [ ]:
class CTesterPRNG(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprng {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
!./testers/grammar_producer_cprng 0 10 10

In [ ]:
CTesterPRNG().run_test().show()

### Can we make the random go faster?

**idea**: Do the random allocation in one place, and use that later.

#### How fast is /dev/random (and variants)?

Using the best block size, and fastest #counts

In [ ]:
with timeit() as t:
    !dd if=/dev/random of=random.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('random.x').st_size/1024/t.runtime, 'kb per second')

In [ ]:
with timeit() as t:
    !dd if=/dev/urandom of=random.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('random.x').st_size/1024/t.runtime, 'kb per second')

In [ ]:
with timeit() as t:
    !dd if=/dev/zero of=io.x bs=1024 count=1000 2>/dev/null
print("throughput=",os.stat('io.x').st_size/1024/t.runtime, 'kb per second')

**Idea**:
* Pre-allocate random bits, and use only as much as necessary.
* Optimize for < 256 bits by using only a single byte at a time.

```c
uint8_t
map(uint8_t to) {
    uint8_t from = rand_region[rand_cursor++];
    if (rand_cursor >= rand_region_size)
        rand_cursor = 0;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}
```

**idea**: Wraparound at 4 GB to avoid comparisons (this did not work as expected.)

In [ ]:
u_max_int = 4096 * 1024 * 1024

(The better idea is to use a pointer to the last element, and increment it rather than use an array address, which is required for this trick.)

### Can we make random faster?

In [ ]:
my_str = '''
#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>
#include <errno.h>
#include <string.h>

uint8_t* rand_region;
void* stack[INT_MAX];

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

uint8_t* rand_region_size = 0;

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_region;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_size = (uint8_t*) (arr+i);
}

int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    uint8_t* rand_region_init;

    char* rand_file = argv[1];
    rand_fd = open(rand_file, O_RDWR | O_CREAT, 0600);
    size_t u_max = (uint64_t)powl(2,32);
    int res = ftruncate(rand_fd, u_max);
    if (res == -1) {
        fprintf(stdout, "Error: %s\\n", strerror(errno));
        return 4;
    }
    rand_region = mmap(0, u_max, PROT_READ| PROT_WRITE, MAP_SHARED, rand_fd, 0);
    rand_region_init = rand_region;
    if (rand_region == (uint8_t*)-1) {
        exit(3);
    }
    initialize_random(u_max);
    msync(rand_region, st.st_size, MS_SYNC);
    munmap(rand_region, st.st_size);
    long rand_size = rand_region_size - rand_region_init;
    ftruncate(rand_fd, rand_size);
    /*fprintf(stdout, "%ld\\n", rand_size);*/
    close(rand_fd);
    return 0;
}
'''

In [ ]:
with open('testers/rand.c', 'w+') as f:
    print(my_str, file=f)

In [ ]:
%cd testers
!cc -g -o rand -Ofast rand.c
%cd ..

In [ ]:
with timeit() as t:
    !./testers/rand random1.x
print("throughput=",os.stat('random1.x').st_size/1024/t.runtime)
!rm random1.x

Allocate a file `u_max_int` size, and mmap it to memory.

```c
uint32_t rand_cursor = 0;
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = rand_region[rand_cursor++];
    return ((uint16_t) from * (uint16_t) to) >> 8;
}
```

Does it help?

In [ ]:
# II
# https://lemire.me/blog/2016/06/27/a-fast-alternative-to-the-modulo-reduction/
class CFuzzerExtRand(CFuzzer):
    def main_hdefs(self):
        s = super().main_hdefs()
        return s + '''
#include <unistd.h>
#include <limits.h>
#include <fcntl.h>
#include <sys/types.h>
#include <sys/mman.h>
#include <sys/stat.h>
#include <math.h>
'''
    
    def fn_map_def(self):
        return '''
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = rand_regionp[rand_cursor++];
    if (rand_cursor >= rand_region_size)
        rand_cursor = 0;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}

static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

uint8_t* rand_region_sizep = 0;

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_regionp;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_sizep = (uint8_t*) (arr+i);
}
'''
    def main_rand_var_defs(self):
        return '''
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_regionp[rand_region_size];
uint64_t rand_cursor = 0;
'''
    
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_rand_var_defs()

    def fuzz_hdefs(self):
        s = super().fuzz_hdefs()
        return s + '''
#include <unistd.h>
#include <stdint.h>'''
    
    def fuzz_rand_var_defs(self):
        return '''
extern uint8_t* rand_regionp;
extern uint64_t rand_cursor;
extern uint64_t rand_region_size;
uint8_t map(uint8_t to);'''
 
    def fn_main_rand_frag(self):
        return '''\
    initialize_random(rand_region_size);
    rand_cursor = seed;
    '''
 
    def fn_main_def(self):
        return '''
int main(int argc, char** argv) {
    struct stat st;
    int max_num, seed, rand_fd, out_fd;
    long out_size;
%(input_frag)s
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % { 'input_frag': self.fn_main_input_frag(),
         'loop_frag': self.fn_main_loop_frag(),
         'rand_frag': self.fn_main_rand_frag()}

In [ ]:
main_src, fuzz_src = CFuzzerExtRand(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprngextr_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprngextr_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_cprngextr_fuzz.c

In [ ]:
!nl -ba testers/grammar_producer_cprngextr_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_cprngextr grammar_producer_cprngextr_main.c grammar_producer_cprngextr_fuzz.c
%cd ..

In [ ]:
# II
!./testers/grammar_producer_cprngextr 0 10 10

In [ ]:
# II
class CTesterPRNGExt(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprngextr {seed} {self.max_num} {max_depth} ./random.x > {fn}"

In [ ]:
CTesterPRNGExt().run_test().show()

In [ ]:
# II
class CFuzzerExtRandP(CFuzzerExtRand):
    def fn_map_def(self):
        return '''
uint8_t
__attribute__((always_inline))
map(uint8_t to) {
    uint8_t from = *rand_regionp++;
    if (rand_regionp >= rand_region_sizep)
        rand_regionp = rand_region_initp;
    return ((uint16_t) from * (uint16_t) to) >> 8;
}


static inline uint64_t rotl(const uint64_t x, int k) {
    return (x << k) | (x >> (64 - k));
}
static uint64_t r__s[4] = {13343, 9838742, 223185, 802124}; /*TODO: initialize with seed.*/
uint64_t
next(void) {
    const uint64_t result_starstar = rotl(r__s[1] * 5, 7) * 9;

    const uint64_t t = r__s[1] << 17;

    r__s[2] ^= r__s[0];
    r__s[3] ^= r__s[1];
    r__s[1] ^= r__s[2];
    r__s[0] ^= r__s[3];

    r__s[2] ^= t;

    r__s[3] = rotl(r__s[3], 45);

    return result_starstar;
}

void
__attribute__((flatten))
initialize_random(uint64_t max_chars) {
    uint64_t* arr = (uint64_t*) rand_regionp;
    uint64_t i;
    for (i=0; i < max_chars/8; i++) { /*max_space/8 because we have 8 bytes*/
        arr[i] = next();
    }
    rand_region_sizep = (uint8_t*) (arr+i);
}
'''
    def main_rand_var_defs(self):
        return '''
uint8_t* rand_region_sizep = 0;
const uint64_t rand_region_size = 1ULL << 16;
uint8_t rand_region_initp[rand_region_size];

uint8_t* rand_regionp = rand_region_initp;
'''
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);
'''
    
    def fn_main_loop_frag(self):
        return '''
    for (int i = 0; i < max_num; i++) {
        gen_init__();
    }
'''
    def fn_main_rand_frag(self):
        return '''\
    initialize_random(rand_region_size);
    rand_regionp += seed;
    '''
    def fn_main_def(self):
        return '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    char* out_region_initp;
    int out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag()
       }

In [ ]:
main_src, fuzz_src = CFuzzerExtRandP(c_grammar).fuzz_src()
with open('testers/grammar_producer_cprngextrP_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_cprngextrP_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_cprngextrP_main.c

In [ ]:
!cat testers/grammar_producer_cprngextrP_fuzz.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_cprngextrP grammar_producer_cprngextrP_main.c grammar_producer_cprngextrP_fuzz.c
%cd ..

In [ ]:
!./testers/grammar_producer_cprngextrP 0 10 10

In [ ]:
# II
class CTesterPRNGExtP(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_cprngextrP {seed} {self.max_num} {max_depth} > {fn}"

In [ ]:
CTesterPRNGExtP().run_test().show()

## Using faster IO.

### MMap

**Idea**:
* `mmap` to a file, write the bits, and `ftruncate()` to the new size.

In [ ]:
# II
class CMMapIOFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
char* out_regionp;
uint64_t out_cursor = 0;
'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}
'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    char* iomax = getenv("IO_LIMIT");
    uint64_t u_iomax = UINT_MAX * 10ULL; // 40G
    if (iomax) {
        u_iomax = 1ULL << atoi(iomax);
    }
    if (argc > 4) {
        out_fd = open(argv[4], O_RDWR | O_CREAT, 0600);
    } else {
        out_fd = open("io.x", O_RDWR | O_CREAT, 0600);
    }
    if (iomax) {
        int res = ftruncate(out_fd, u_iomax);
        if (res != 0) {
            perror("truncate failed");
            exit(2);
        }
    } else {
        int res = try_truncate(out_fd);
        if (res < 32) {
            perror("truncate failed");
            fprintf(stderr,"%d\\n", res);
            exit(5);
        }
    }
    fstat(out_fd, &st);
    out_regionp = mmap(0, st.st_size, PROT_READ|
                      PROT_WRITE, MAP_SHARED, out_fd, 0);
    if (out_regionp == (caddr_t)-1) {
        exit(3);
    }
    '''
    
    def fn_main_sync_frag(self):
        return '''
    msync(out_regionp, st.st_size, MS_SYNC);
    munmap(out_regionp, st.st_size);
    ftruncate(out_fd, out_cursor);
    close(out_fd);
'''

    def fn_truncateio(self):
        return '''
#include <errno.h>
int try_truncate(int fd) {
    for (off_t len = 63; len > 0; len--) {
      uint64_t m = 1ULL << len;
      errno = 0;
      int ret = ftruncate(fd, m);
      if (ret == 0) {
        return len;
      }
    }
    return -1;
}
'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [ ]:
main_src, fuzz_src = CMMapIOFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_mmapio_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_mmapio_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_mmapio_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_mmapio grammar_producer_mmapio_main.c grammar_producer_mmapio_fuzz.c
%cd ..

In [ ]:
# II
!./testers/grammar_producer_mmapio 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
# II
class CTesterMMap(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_mmapio {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterMMap().run_test().show()

### FWrite

In [ ]:
class CFWriteFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
const uint64_t size = UINT_MAX; /*max size of a single input -- 4G*/
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;
FILE* fs;
'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}'''
    
    def fuzz_out_var_defs(self):
        return '''
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    if (argc > 4) {
        out_fd = open(argv[4], O_RDWR | O_CREAT, 0600);
    } else {
        out_fd = open("io.x", O_RDWR | O_CREAT, 0600);
    }
    fs = fdopen(out_fd, "w");
'''

    def fn_main_sync_frag(self):
        return '''
    fclose(fs);
    close(out_fd);
'''

    def fn_truncateio(self):
        return '''
'''
    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__();
        fwrite(out_regionp, sizeof(char), out_cursor, fs);
        out_cursor = 0;
    }
'''

    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [ ]:
main_src, fuzz_src = CFWriteFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_fwrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_fwrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_fwrite_fuzz.c

In [ ]:
!cat testers/grammar_producer_fwrite_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_fwrite grammar_producer_fwrite_main.c grammar_producer_fwrite_fuzz.c
%cd ..

In [ ]:
# II
!./testers/grammar_producer_fwrite 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
# II
class CTesterFWrite(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_fwrite {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterFWrite().run_test().show()

### No output 

In [ ]:
class CNoWriteFuzzer(CFuzzerExtRandP):
    def main_out_var_defs(self):
        return '''
const uint64_t size = UINT_MAX; // size of a single output item -- 4G
char out_region_initp[size];
char *out_regionp = out_region_initp;
uint64_t out_cursor = 0;'''
    def main_var_defs(self):
        s = super().main_var_defs()
        return s + self.main_out_var_defs()
     
    def fn_out_def(self):
        return '''
void
__attribute__((always_inline))
out(char c) {
    out_regionp[out_cursor++] = c;
}'''
    
    def fuzz_out_var_defs(self):
        return '''\
void out(char c);
extern char* out_regionp;
extern uint64_t out_cursor;
'''

    def fn_main_input_frag(self):
        return '''
    if (argc < 3) {
        printf("%s <seed> <max_num> <max_depth>\\n", argv[0]);
        return 0;
    }
    seed = atoi(argv[1]);
    max_num = atoi(argv[2]);
    max_depth = atoi(argv[3]);'''

    def fn_main_out_frag(self):
        return '''
    '''
    
    def fn_main_sync_frag(self):
        return '''
    '''

    def fn_truncateio(self):
        return '''
        '''
    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        gen_init__();
        // throw away
        out_size += out_cursor;
        out_cursor = 0;
    }
    printf("%lld\\n", out_size);
    '''

    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    int seed, max_num;
%(input_frag)s
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'out_frag': self.fn_main_out_frag(),
        'sync_frag': self.fn_main_sync_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag(),
       }

In [ ]:
main_src, fuzz_src = CNoWriteFuzzer(c_grammar).fuzz_src()
with open('./testers/grammar_producer_nowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('./testers/grammar_producer_nowrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_nowrite grammar_producer_nowrite_main.c grammar_producer_nowrite_fuzz.c
%cd ..

In [ ]:
# II
!./testers/grammar_producer_nowrite 0 10 10

In [ ]:
# II
class CTesterNoWrite(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_nowrite {seed} {self.max_num} {max_depth} > {fn}"
  
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWrite().run_test().show()

## Fuzzer as a VM

### MMap

#### Direct threaded VM

In [ ]:
# II
class DTMMapFuzzer(CMMapIOFuzzer):
    def fn_out_def(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                res.append('''\
    *out_regionp++ = '%s';''' % self.esc_char(token))
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)
    
    def main_out_var_defs(self):
        return'''
char* out_regionp;
int out_cursor;
'''
    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    for(int i=0; i < max_num; i++) {
        gen_init__(max_depthp);
        *out_regionp++ = '\\n';
    }
    *out_regionp = 0;'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    char* out_region_initp;
    int rand_fd, out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
    out_region_initp = out_regionp;
    out_region_sizep = out_regionp + st.st_size;
%(loop_frag)s
    out_size = out_regionp - out_region_initp;
    out_cursor = out_size;
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [ ]:
main_src, fuzz_src = DTMMapFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_dtmmap_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_dtmmap_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!wc -l testers/grammar_producer_dtmmap_fuzz.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_dtmmap grammar_producer_dtmmap_main.c grammar_producer_dtmmap_fuzz.c
%cd ..

In [ ]:
# II
!./testers/grammar_producer_dtmmap 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
# II
class CTesterMMapDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_dtmmap {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterMMapDT().run_test().show()

#### Context threaded VM

In [ ]:
# II
class CTMMapFuzzer(DTMMapFuzzer):
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':repr(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': repr(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':repr(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctmmap_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''\
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''\
    for(int i=0; i < max_num; i++) {
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
    }
    *out_regionp = 0;'''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_initp;
    int out_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(out_frag)s
    out_region_initp = out_regionp;
%(loop_frag)s
    out_size = out_regionp - out_region_initp;
    out_cursor = out_size;
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [ ]:
vm_ops, main_src, fuzz_src = CTMMapFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctmmap_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctmmap_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctmmap_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [ ]:
!nl -ba testers/grammar_producer_ctmmap_fuzz.s

In [ ]:
!nl -ba testers/grammar_producer_ctmmap_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctmmap grammar_producer_ctmmap_main.c grammar_producer_ctmmap_fuzz.s
%cd ..

In [ ]:
# II
!./testers/grammar_producer_ctmmap 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
# II
class CTesterMMapCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctmmap {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterMMapCT().run_test().show()

### FWrite

#### Direct threaded VM

In [ ]:
# II
class CFWriteDTFuzzer(CFWriteFuzzer):
    def fn_out_def(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                t = self.esc_char(token)
                res.append('''\
    *out_regionp++ = '%s';''' % t)
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    *out_regionp++ = '\\n';
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)

    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    fs = fdopen(out_fd, "w");
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depthp);
        out_cursor = out_regionp - out_region_initp;
        fwrite(out_region_initp, sizeof(char), out_cursor, fs);
    }
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    char* out_region_sizep = 0;
    int out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [ ]:
main_src, fuzz_src = CFWriteDTFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_fwritedt_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_fwritedt_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!wc -l testers/grammar_producer_fwritedt_fuzz.c

In [ ]:
!cat testers/grammar_producer_fwritedt_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_fwritedt grammar_producer_fwritedt_main.c grammar_producer_fwritedt_fuzz.c
%cd ..

In [ ]:
# II
class CTesterFWriteDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_fwritedt {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
# II
!./testers/grammar_producer_fwritedt 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
CTesterFWriteDT().run_test().show()

#### Context threaded VM

In [ ]:
# II
class CFWriteCTFuzzer(CFWriteDTFuzzer):
    
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':self.esc(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': self.esc(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':self.esc(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctfwrite_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''
    fs = fdopen(out_fd, "w");
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
        out_cursor = out_regionp - out_region_initp;
        fwrite(out_region_initp, sizeof(char), out_cursor, fs);
    }
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    long out_size;
    int out_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [ ]:
vm_ops, main_src, fuzz_src = CFWriteCTFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctfwrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctfwrite_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctfwrite_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [ ]:
!nl -ba testers/ctfwrite_vm_ops.s

In [ ]:
!wc -l testers/grammar_producer_ctfwrite_fuzz.s

In [ ]:
!cat testers/grammar_producer_ctfwrite_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctfwrite grammar_producer_ctfwrite_main.c grammar_producer_ctfwrite_fuzz.s
%cd ..

In [ ]:
# II
!./testers/grammar_producer_ctfwrite 0 10 10 io.x

In [ ]:
!cat io.x

In [ ]:
class CTesterFWriteCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctfwrite {seed} {self.max_num} {max_depth} {fn}"

In [ ]:
CTesterFWriteCT().run_test().show()

### No Output

#### Direct threaded VM

In [ ]:
class DTNoWriteFuzzer(CNoWriteFuzzer):
    def fn_out_def(self): return ''
    def fn_main_out_frag(self): return ''
    def gen_rule_src(self, rule, k, j):
        res = []
        leaf = True
        for i, token in enumerate(rule):
            if token in self.grammar:
                leaf = False
                trules = self.grammar[token] # ordered by cost
                len_min_choices = len(self.c_grammar[token])
                assert len(trules) < 256
                cheap_strings = self.pool_of_strings[token]
                if len(cheap_strings) < 256: # we only have 255 random choices
                    check_pool = '''
        val = map(%(len_cheap_strings)s);
        const char* str = pool_%(k)s[val];
        const int str_l = pool_l_%(k)s[val];
        for (int i = 0; i < str_l; i++) {
            *out_regionp++ = str[i];
        }
        --returnp;
        goto **returnp; 
            ''' % { 'len_cheap_strings': len(cheap_strings), 'k': self.k_to_s(token)}
                else:
                    check_pool = '''
        val = map(%(len_min_choices)s);
                ''' % {'len_min_choices':len_min_choices}
                res.append('''\
    *returnp = &&return__%(i)d__%(j)d__%(k)s;
    if (returnp > max_depthp) {
        %(check_pool)s;
    } else {
        val = map(%(len_rules)s);
    }
    goto *gen_%(t)s[val];
return__%(i)d__%(j)d__%(k)s:;
            ''' % {'i':i, 'j':j, 'k':self.k_to_s(k),
                   't':self.k_to_s(token), 'rnum':0, 'len_rules':len(trules), 'len_min_choices':len_min_choices, 'check_pool':check_pool})
            else:
                t = self.esc_char(token)
                res.append('''\
    *out_regionp++ = '%s';''' % t)
        return res, leaf
    
    def gen_alt_src_1rule(self, k):
        rule = self.grammar[k][0]
        ri = 0
        src, leaf = self.gen_rule_src(rule, k, ri)
        body = '\n'.join(src)
        result = []
        if leaf:
            return '''
gen_%(name)s_0: {
%(body)s
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}
        else:
             return '''
gen_%(name)s_0: {
    ++returnp;
    // single -- no switch
%(body)s
    --returnp;
    goto **returnp;
}''' % {'name':self.k_to_s(k), 'body':body}

    def gen_alt_src(self, k):
        rules = self.grammar[k]
        ret = self.k_to_s(k)
        result = []
        if len(rules) == 1: return self.gen_alt_src_1rule(k)
        for ri, rule in enumerate(rules):
            src, leaf = self.gen_rule_src(rule, k, ri)
            body = '\n'.join(src)
            if leaf:
                result.append('''
gen_%(name)s_%(rnum)d: {
%(body)s
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
            else:
                 result.append('''
gen_%(name)s_%(rnum)d: {
    ++returnp;
%(body)s
    --returnp;
    goto **returnp;
}
    ''' % {'name': self.k_to_s(k), 'rnum': ri, 'body':body})
        return '\n'.join(result)

    def fuzz_out_var_defs(self):
        return '''\
extern char* out_regionp;'''
    
    def fuzz_rand_var_defs(self):
        return '''
uint8_t map(uint8_t to);'''
    
    def fuzz_stack_var_defs(self):
        return '''
extern void* stackp[];
'''

    def fuzz_entry(self):
        result = ['''
void gen_init__(void** max_depthp) {
    uint8_t val;
    void** returnp = stackp;
    *returnp =  &&return__init;
''']
        for k in self.grammar:
            l = []
            for ri,rule in enumerate(self.grammar[k]):
                l.append('&&gen_%(k)s_%(ri)d' % {'k':self.k_to_s(k), 'ri':ri})
            s = '''
    void** gen_%(k)s[] = {
%(body)s
    };''' % {'k': self.k_to_s(k), 'body': ',\n'.join(l)}
            result.append(s)
        result.append('''
    goto gen_start_0;''')
        result.append(self.fuzz_fn_defs())
        result.append("""
return__init:
    *out_regionp++ = '\\n';
    return;
return_abort:
    exit(10); 
}""")
        return '\n'.join(result)
    
    def main_stack_var_defs(self):
        return'''
int max_depth;
void** max_depthp;
void* stackp[INT_MAX];
'''
    def main_init_var_defs(self):
        return'''
void gen_init__(void** max_depthp);
'''

    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depthp);
        out_cursor = out_regionp - out_region_initp;
        out_size += out_cursor;
    }
    printf("%lld\\n", out_size);
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    char* out_region_sizep = 0;
    int out_fd;
    int seed, max_num;
%(input_frag)s
    max_depthp = stackp + max_depth;
%(rand_frag)s
%(out_frag)s
%(loop_frag)s
%(sync_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'out_frag': self.fn_main_out_frag(),
        'loop_frag': self.fn_main_loop_frag(),
        'sync_frag': self.fn_main_sync_frag()
       }


    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_hdefs(),
                          self.fuzz_var_defs(),
                          self.fn_fuzz_decs(),
                          self.string_pool_defs(),
                          # self.fuzz_fn_defs(),
                          self.fuzz_entry()])

In [ ]:
main_src, fuzz_src = DTNoWriteFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_dtnowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_dtnowrite_fuzz.c', 'w+') as f:
    print(fuzz_src, file=f)

In [ ]:
!cat testers/grammar_producer_dtnowrite_main.c

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_dtnowrite grammar_producer_dtnowrite_main.c grammar_producer_dtnowrite_fuzz.c
%cd ..

In [ ]:
!./testers/grammar_producer_dtnowrite  0 10 10

In [ ]:
class CTesterNoWriteDT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_dtnowrite {seed} {self.max_num} {max_depth} > {fn}"
  
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWriteDT().run_test().show()

#### Context threaded VM

In [ ]:
class CTNoWriteFuzzer(DTNoWriteFuzzer):
    def fn_main_out_frag(self): return ''
    def fn_main_sync_frag(self): return ''
    
    def fn_choice(self, val):
        return '''
    # [ random 
    # extract one byte from the random stream %%r14,
    movq (%%r14), %%rdi
    # advance the random cursor
    inc %%r14                                     # rand_region++
    movzbl %%dil, %%edi                           # X  --- (rdi:(edi:(di:(dil))))
    # then multiply with the choices we have

    xor %%rsi, %%rsi                              # avoid data dependencies
    movb $%(val)s, %%sil                          # = %(val)s       
    movzbl %%sil, %%edx
    imull %%edi, %%edx                            # m = (short) x * (short) N)
    sarl $8, %%edx                                # return (char)(m >> 8) ;
    # random ]
    # %%edx now contains the selected random value from %(val)d options''' % {'val':val}

    def cheap_strings(self, k):
        cheap_strings = self.pool_of_strings[k]
        results = ['''
    # --- cheap -- [''']
        results.append('''
%(choices)s
''' % {'choices':self.fn_choice(len(cheap_strings)), 'len_choices': len(cheap_strings)})
        # get the choices from vm, then call it, and return.
        
        results.append('''
    # now we have the right print quad in %%edx. Load the right address and call it.
    leaq _%(key)s_prints(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ret
    ''' % {'key': self.k_to_s(k)})
        results.append('''
    # --- cheap -- ]''')
        return '\n'.join(results)
    
    def output_char(self, c):
        if len(c) != 1:
            assert c[0] == '\\'
            c = c[-1]
        return '''
   movb $%(ichar)d, (%%r13)                     # '%(char)s'
   inc %%r13                                    # out_region++   : increment a byte (r13++)
   ''' % {'char':self.esc(c), 'ichar':ord(c)}

    def gen_rule_src(self, rule, k, j):
        # in each rule, there are a number of tokens.
        # iter each token in turn, choose the right rule and call.
        result = []
        for token in rule:
            if token not in self.grammar:
                result.append(self.output_char(token))
                continue
            else:
                # how many choices do we have?
                rules = self.grammar[token]
                result.append('''
    # start the choice machine.
    # length of rules = %(len_rules)d
%(choices)s
    # --- switch ---
    ''' % {'choices': self.fn_choice(len(rules)), 'len_rules':len(rules)})
                result.append('''
    # now we have the right choice in %%edx. Load the right address and call it.
    leaq _%(key)s_choices(%%rip), %%rcx
    leaq (%%rcx, %%rdx, 8), %%rax
    callq *(%%rax)
    ''' % {'key': self.k_to_s(token)})
        return '\n'.join(result)

    def gen_alt_src(self, k):
        result = []
        for ruleid, rule in enumerate(self.grammar[k]):
            # produce a skeletal subroutine structure.
            result.append('''
gen_%(key)s_%(ruleid)s:
    # check if the max depth is breached.
    cmpq %%rsp, %%r8                             # returnp(rbp) <> max_depth(r8) ?
    jle _%(key)s_%(ruleid)s_fi                       # returnp <= max_depth
    
%(return_cheap_string)s
_%(key)s_%(ruleid)s_fi:
''' % {'return_cheap_string': self.cheap_strings(k),
       'key':self.k_to_s(k),
       'ruleid':ruleid,
       'last_label':self.last_label})
            self.last_label += 1
            result.append(self.gen_rule_src(rule, k, ruleid))
            # we were called. So simply return.
            result.append('''
    ret
            ''')
        return '\n'.join(result)
 
    def fn_fuzz_decs(self):
        result = ['''
  .section  __DATA,__data

# Virtual Machine OPS.
        ''']
        for k in self.grammar:
            result.append('''
    .globl  _%(key)s_choices
    .p2align 4
_%(key)s_choices:''' % {'key':self.k_to_s(k)})
            for i, rule in enumerate(self.grammar[k]):
                result.append('''\
    .quad gen_%s_%d''' % (self.k_to_s(k), i))
                
        for k in self.pool_of_strings:
            result.append('''
    .globl  _%(key)s_prints
    .p2align 4
_%(key)s_prints:''' % {'key':self.k_to_s(k)})
            for string in self.pool_of_strings[k]:
                result.append('''\
    .quad %s''' % (self.all_prints[string]))
                
                
        result.append('''
# End Virtual Machine OPS.''')
        return '\n'.join(result)

    def gen_cheap(self, grammar):
        all_strings = set()
        for k in grammar:
            all_strings |= set(self.pool_of_strings[k])
        all_strings = list(all_strings)
        all_strings.sort(key=lambda item: (-len(item), item))
        all_prints_hash = {}
        result = ['''
.text
        ''']
        for i, s_ in enumerate(all_strings):
            s = s_
            result.append('''\
print_%(name)d: # "%(value)s"''' % {'name': i, 'value': self.esc(s)})
            for j in s:
                result.append('''\
    movb $%(ichar)s, (%%r13)            # '%(char)s'
    inc %%r13''' % {'ichar':ord(j), 'char':self.esc(j)})
            result.append('''\
    ret''')
            all_prints_hash[s_] = 'print_%d' % i
        return ('\n'.join(result), all_prints_hash)
 
    def fuzz_entry(self):
        result = ["""
#include "ctnowrite_vm_ops.s"
.macro pushaq
    push %%rsp
    push %%rbp
    push %%r8
    push %%r9
    push %%r10
    push %%r11
    push %%r12
    push %%r13
    push %%r14
    push %%r15
.endm


.macro popaq
    pop %%r15
    pop %%r14
    pop %%r13
    pop %%r12
    pop %%r11
    pop %%r10
    pop %%r9
    pop %%r8
    pop %%rbp
    pop %%rsp
.endm

.global %(os)sgen_init__
.global return__init
.text
%(os)sgen_init__:
    # 1 rdi = max_depth
    # 2 rsi = returnp
    # 3 rdx = &out_region
    # 4 rcx = &rand_region
    pushaq

    leal 0(,%%rdi,8), %%eax
    movq %%rsp, %%r8
    subq %%rax, %%r8

    movq %%rdx, %%r11                              # &out_region
    movq %%rcx, %%r12                              # &rand_region
    movq (%%r11),%%r13                             # out_region
    movq (%%r12),%%r14                             # rand_region

    # general regs
    # rax, rcx, rdx, rbx, rsi,rdi
    # rbp, r8-r15
    
    call gen_start_0
    movq %%r13, (%%r11)                            # *(&out_region) <-
    movq %%r14, (%%r12)                            # *(&rand_region) <-
    popaq
    movq  $0, %%rax
    ret   
""" % {'os': '_' if sys.platform == 'darwin' else ''}]
        result.append(self.fuzz_fn_defs())
        return ''.join(result)

    def main_init_var_defs(self):
        return'''
void gen_init__(uint32_t max_depth, void** returnp, char** out_region, uint8_t** rand_region);
'''

    def fn_main_loop_frag(self):
        return '''
    uint64_t out_size = 0;
    for(int i=0; i < max_num; i++) {
        out_regionp = out_region_initp;
        gen_init__(max_depth32, stackp, &out_regionp, &rand_regionp);
        *out_regionp++ = '\\n';
        out_cursor = out_regionp - out_region_initp;
        out_size += out_cursor;
    }
    printf("%lld\\n", out_size);
    '''
    
    def fn_main_def(self):
        return self.fn_truncateio() + '''
int main(int argc, char** argv) {
    struct stat st;
    int rand_fd;
    uint32_t max_depth32;
    int seed, max_num;
%(input_frag)s
    max_depth32 = max_depth;
%(rand_frag)s
%(loop_frag)s
    return 0;
}''' % {'input_frag': self.fn_main_input_frag(),
        'rand_frag': self.fn_main_rand_frag(),
        'loop_frag': self.fn_main_loop_frag()
       }
    
    def fuzz_src(self, key='<start>'):
        self.last_label = 0
        self.cheap, self.all_prints = self.gen_cheap(self.grammar)
        ext_strings = '\n'.join([self.fn_fuzz_decs(), self.cheap])
        return ext_strings, self.gen_main_src(), self.gen_fuzz_src()
    
    def gen_fuzz_src(self):
        return '\n'.join([self.fuzz_entry()])

In [ ]:
vm_ops, main_src, fuzz_src = CTNoWriteFuzzer(c_grammar).fuzz_src()
with open('testers/grammar_producer_ctnowrite_main.c', 'w+') as f:
    print(main_src, file=f)
with open('testers/grammar_producer_ctnowrite_fuzz.s', 'w+') as f:
    print(fuzz_src, file=f)
with open('testers/ctnowrite_vm_ops.s', 'w+') as f:
    print(vm_ops, file=f)

In [ ]:
%cd testers
!cc -g -Ofast -o grammar_producer_ctnowrite grammar_producer_ctnowrite_main.c grammar_producer_ctnowrite_fuzz.s
%cd ..

In [ ]:
!./testers/grammar_producer_ctnowrite 0 10 10

In [ ]:
class CTesterNoWriteCT(CTester):
    def exec_program(self, seed, max_depth, t):
        fn = self.ofile(max_depth, seed)
        return f"./testers/grammar_producer_ctnowrite {seed} {self.max_num} {max_depth} > {fn}"
 
    def post_time(self):
        super().post_time()
        with open(self.file) as f:
            self.size = int(f.read())

In [ ]:
CTesterNoWriteCT().run_test().show()

# Results

In [ ]:
for k in TX:
    print(k)
    for depth in TX[k]:
        print(depth)
        if 'avgruntime' not in TX[k][depth]: continue
        print('\truntime =',TX[k][depth]['avgruntime'])
        print('\tsize = ',TX[k][depth]['avgsize'])
        print('\tthroughput =',TX[k][depth]['avgthroughput'])
    print()
    
END_TIME = datetime.now()

In [ ]:
str(END_TIME - START_TIME)

In [ ]:
import json

In [ ]:
!mkdir -p results

In [ ]:
from datetime import datetime
curtime = datetime.now().isoformat()
name = 'results/tx-%s.json' % curtime
with open(name, 'w+') as f:
    print(json.dumps(TX), file=f)
print(name)

# Grammar Transformations

## Speed vs Code size Tradeoffs

A sliding scale of

* Completely String pools
* Compile to a state machine (CFG to Regular expression of fixed depth)
* Encode depth in function name (Remove depth comparisons)

### Expanding the use of string pools from just closing to before max_depth is exhausted

 We are not stuck with a pool of strings only after exhaustion of max_depth. But we will have to account for differing probabilties of different strings if we want to achieve a distribution of strings as the original. Whether it is required to have the same distribution as the original is a different question (because the original is clearly non-optimial -- shallow paths have more chance of being explroed again).

## Inlining

For inlining, we simply iterate through each key in the grammar, and each rule corresponding to a single key. For each rule, we inline one level, which will give us a list of corresponding rules. This set of rules will replace the original rule for the key.

# Probabilistic Fuzzing

One of the problems with our dumb grammar fuzzer is that each alternative rule for a key expansion is given the same probability. Hence, given a JSON element that can be a boolean, number, object or an array, the boolean (true and false) values will occur very 5 elements. This is clearly non-optimal. Hence, we need to extend our fuzzer to include probabilities in the grammar definition.

Once we adopt probabilistic fuzzing, we can generate a probabilistic profile of the grammar rules by expanding the grammar to a given depth, and simply counting the number of complete items produced by each expansion. This can ensure that there is a high probability of exploring at least to that depth.

In [ ]:
from functools import reduce
import operator

def items_in_rule(grammar, rule, depth, max_depth):
    if depth > max_depth: return 1
    return reduce(operator.mul,
                  [items_in_key(grammar, key, depth+1, max_depth)
                   for key in  rule], 1)


def items_in_key(grammar, key, depth=0, max_depth=10):
    if key not in grammar: return 1
    return sum(items_in_rule(grammar, rule, depth, max_depth)
               for rule in grammar[key])

def explore_grammar(grammar, max_depth):
    new_g = {}
    for k in grammar:
        new_rules = []
        for rule in grammar[k]:
            items = items_in_rule(grammar, rule, depth=0, max_depth=max_depth)
            new_rule = (rule, items)
            new_rules.append(new_rule)
        new_g[k] = new_rules
    return new_g

def to_ranges(pgrammar):
    new_g = {}
    for k in pgrammar:
        last = 0
        elts = []
        for elt in pgrammar[k]:
            rule, count = elt
            frm = last
            last += count
            to = last
            new_elt = rule, (frm, to)
            elts.append(new_elt)
        new_g[k] = elts
    return new_g
p_grammar = explore_grammar(my_grammar, 1)

In [ ]:
p_grammar

In [ ]:
def get_included_rule(idx, rules_):
    for r,rng in rules_:
        if rng[0] <= idx and idx < rng[1]: return r
    assert False

def gen_key(grammar, key, depth=0, max_depth=10):
    if key not in grammar: return [key]
    if depth > max_depth: return [random.choice(pool_of_strings[key])]
    
    rules_ = grammar[key]
    max_val = max([j for rule, (i,j) in rules_])
    rule = get_included_rule(random.randrange(max_val), rules_)
    return gen_rule(grammar, rule, depth+1, max_depth)

def gen_rule(grammar, rule, depth, max_depth):
    return sum([gen_key(grammar, token, depth, max_depth) for token in rule], [])

def grammar_producer_p(grammar, key='<start>'):
    cp_grammar = to_ranges(grammar)
    return ''.join(gen_key(cp_grammar, key))

In [ ]:
pf = PooledFuzzer(my_grammar)
pool_of_strings = pf.pool_of_strings
for i in range(10):
    print(grammar_producer_p(p_grammar))

# Generating Large Inputs Fast